# Loading some packages

In [1]:
import numpy as np
from numba import njit
import pandas as pd
import polars as pl
from pathlib import Path
from collections import defaultdict
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

# Some supplementary functions

In [2]:
def generate_fold_data(data_path, fold=1):

    df = pd.read_csv(data_path)
    df['datetime'] = pd.to_datetime(df['datetime'])
    df = df.iloc[50400*(fold-1):fold*50400]
    df.sort_values('datetime', ascending=True, inplace=True)
    df.reset_index(inplace=True, drop=True)

    return df

@njit(cache=True)
def get_signals(signal_list, exit_list):

    start_idx = 0
    exit_idx = 0
   
    for i in range(len(signal_list[0])):

        if i == start_idx:

            if signal_list[0][i] == 0:

                start_idx += 1

                exit_list[0][i] = 0

            else:

                for j in range(i+1, len(exit_list[0])):
                    if exit_list[0][j] == -signal_list[0][i]:
                        exit_idx = j
                        break
                    else:
                        exit_idx = j

                for k in range(i+1, exit_idx+1):
                    if signal_list[0][k] == -signal_list[0][i]:
                        exit_idx = k
                        exit_list[1][k] = signal_list[1][k]
                        break
                    else:
                        exit_idx = k

                for p in range(i+1, exit_idx):
                    signal_list[0][p] = 0
                    exit_list[0][p] = 0
                
                if exit_idx == len(signal_list[0]) - 1 and exit_list[0][exit_idx] != -signal_list[0][i]:
                    exit_list[0][exit_idx] = 0
                    exit_list[0][i] = 0
                    signal_list[0][exit_idx] = 0
                else:
                    exit_list[0][exit_idx] = -signal_list[0][i]
                    exit_list[0][i] = 0
                    signal_list[0][exit_idx] = 0
                
                start_idx = exit_idx + 1

        else:

            continue

    if sum(np.abs(signal_list[0])) == sum(np.abs(exit_list[0])):

        return signal_list, exit_list
    
    else:

        for i in range(len(signal_list[0])):
            if signal_list[0][-(i+1)] != 0:
                signal_list[0][-(i+1)] = 0
                break

        return signal_list, exit_list
    
@njit(cache=True)
def create_position_open_prices(signal_list, exit_list):

    pos_open_prices = np.zeros(len(signal_list[0]))
    pos_exit_prices = np.zeros(len(exit_list[0]))

    start_idx = 0
    price_idx = 0

    for i in range(len(signal_list[0])):
        if exit_list[0][i] != 0:
            for j in range(start_idx, i):
                if signal_list[0][j] != 0:
                    price_idx = j
                    break
            pos_open_prices[i] = signal_list[1][price_idx]
            pos_exit_prices[i] = exit_list[1][i]
            start_idx = i
        else:
            pass

    return pos_open_prices, pos_exit_prices

@njit(cache=True)
def get_pnl_testing(
    trade_close_prices,
    signal_list, 
    trade_open_prices,
    commission=0.015,
    slippage=0.05,
    init_inv=20000,
    trade_size=0.1
):

    pnl_list = np.zeros(len(trade_close_prices))

    for i in range(len(trade_close_prices)):

        if signal_list[i] == 0 or trade_open_prices[i] == 0:
            pass
        
        # signal_list contains the points where exit occurs
        elif signal_list[i] == -1: 
            temp_n_assets = int(init_inv * trade_size / trade_open_prices[i])
            temp_pnl = temp_n_assets * (trade_close_prices[i] - trade_open_prices[i] * (1 + slippage)) 
            temp_pnl = temp_pnl * (1 - commission)
            init_inv += temp_pnl

        else:
            temp_n_assets = int(init_inv * trade_size / trade_open_prices[i])
            temp_pnl = temp_n_assets * (trade_open_prices[i] * (1 - slippage) - trade_close_prices[i])
            temp_pnl = temp_pnl * (1 - commission)
            init_inv += temp_pnl

        pnl_list[i] = temp_pnl

    return pnl_list

@njit(cache=True)
def get_exit_entry_testing1(
    close_prices, 
    open_prices,  
    signal_list,
    stoploss_th,
    takeprofit_th, 
    commission, 
    slippage, 
    init_inv, 
    trade_size
):

    exit_list = np.zeros((2, len(close_prices)))

    for i in range(len(close_prices)-1):

        if signal_list[1][i] == 0:

            pass

        else:

            temp_n_assets = int(init_inv * trade_size / signal_list[1][i])
            if signal_list[0][i] == 1:
                temp_pnl = temp_n_assets * (close_prices[i] - signal_list[1][i] * (1 + slippage))
            else:
                temp_pnl = -temp_n_assets * (close_prices[i] - signal_list[1][i] * (1 - slippage))
            temp_pnl = temp_pnl * (1 - commission)
            init_inv += temp_pnl

            if -temp_pnl >= stoploss_th or temp_pnl >= takeprofit_th:
                exit_list[0][i+1] = -signal_list[0][i]
                exit_list[1][i+1] = open_prices[i+1]
            else:
                pass
        
    return exit_list

@njit(cache=True)
def get_exit_entry_testing2( 
    open_prices,  
    signal_list,
    n_exit_bars
):

    exit_list = np.zeros((2, len(signal_list[0])))

    n_exit_bars = np.int64(n_exit_bars)

    for i in range(len(signal_list[0])-1):

        if signal_list[0][i] == 0:

            pass

        else:
            
            if i + n_exit_bars < len(signal_list[0]):
                exit_list[0][i+n_exit_bars] = -signal_list[0][i]
                exit_list[1][i+n_exit_bars] = open_prices[i+n_exit_bars]
            else:
                pass
        
    return exit_list

@njit(cache=True)
def get_exit_entry_testing3( 
    open_prices,  
    signal_list
):

    exit_list = np.zeros((2, len(signal_list[0])))

    for i in range(len(signal_list[0])-1):

        if signal_list[0][i] == 0:

            pass

        else:

            for j in range(i+1, len(signal_list[0])):
                if signal_list[0][j] != 0:
                    j = j - 1
                    break
                else:
                    if np.random.rand() > 0.5:
                        break
            
            exit_list[0][j] = -signal_list[0][i]
            exit_list[1][j] = open_prices[j]
        
    return exit_list

def calculate_mean_win_perc_entry_testing(exec_dict, df):

    commission = exec_dict['COMMISSION']
    slippage = exec_dict['SLIPPAGE'] 
    init_inv = exec_dict['AVAILABLE_CAPITAL']
    trade_size = exec_dict['TRADE_SIZE'] 

    if len(exec_dict['buy_idxs']) != 0:
        signal_idxs = list(exec_dict['buy_idxs'])
        if len(exec_dict['sell_idxs']) != 0:
            signal_idxs.extend(list(exec_dict['sell_idxs']))
    else:
        if len(exec_dict['sell_idxs']) != 0:
            signal_idxs = list(exec_dict['sell_idxs'])
        else:
            signal_idxs = []

    if len(signal_idxs) != 0:
        signal_idxs = sorted(signal_idxs)
        signal_idxs_true = [i - 1 for i in signal_idxs]
    else:
        signal_idxs_true = []

    df['buy'] = 0
    if len(exec_dict['buy_idxs']) != 0:
        buy_idxs = [i - 1 for i in list(exec_dict['buy_idxs'])]
        df.loc[df.index.isin(buy_idxs), 'buy'] = 1

    df['sell'] = 0
    if len(exec_dict['sell_idxs']) != 0:
        sell_idxs = [i - 1 for i in list(exec_dict['sell_idxs'])]
        df.loc[df.index.isin(sell_idxs), 'sell'] = -1

    df['signal'] = df['buy'] + df['sell']

    df['new_signal'] = 0
    df['signal_prices'] = 0
    if len(signal_idxs) != 0:
        df.loc[df.index.isin(signal_idxs_true), 'new_signal'] = df.loc[df.index.isin(signal_idxs_true), 'signal'].values
        df.loc[df.index.isin(signal_idxs), 'signal_prices'] = df.loc[df.index.isin(signal_idxs), 'btc_open'].values

    signal_list = np.zeros((2, df.shape[0]))
    signal_list[0][1:] = df['new_signal'].values[:-1]
    signal_list[1] = df['signal_prices'].values

    # fixed stop and target exit testing
    exit_list = get_exit_entry_testing1(
        close_prices=df['btc_close'].values, 
        open_prices=df['btc_open'].values,  
        signal_list=signal_list,
        stoploss_th=50,
        takeprofit_th=100,
        commission=commission, 
        slippage=slippage, 
        init_inv=init_inv, 
        trade_size=trade_size
    )

    signal_list, exit_list = get_signals(signal_list, exit_list)
    pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

    pnl_list = get_pnl_testing(
        trade_close_prices=pos_exit_prices,
        signal_list=exit_list[0], 
        trade_open_prices=pos_open_prices,
        commission=commission, 
        slippage=slippage, 
        init_inv=init_inv, 
        trade_size=trade_size
    )

    fixed_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)

    # fixed bar exit testing
    exit_list = get_exit_entry_testing2( 
        open_prices=df['btc_open'].values,  
        signal_list=signal_list,
        n_exit_bars=5
    )

    signal_list, exit_list = get_signals(signal_list, exit_list)
    pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

    pnl_list = get_pnl_testing(
        trade_close_prices=pos_exit_prices,
        signal_list=exit_list[0], 
        trade_open_prices=pos_open_prices,
        commission=commission, 
        slippage=slippage, 
        init_inv=init_inv, 
        trade_size=trade_size
    )

    fixed_bar_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)

    # random exit testing
    exit_list = get_exit_entry_testing3( 
        open_prices=df['btc_open'].values,  
        signal_list=signal_list
    )

    signal_list, exit_list = get_signals(signal_list, exit_list)
    pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

    pnl_list = get_pnl_testing(
        trade_close_prices=pos_exit_prices,
        signal_list=exit_list[0], 
        trade_open_prices=pos_open_prices,
        commission=commission, 
        slippage=slippage, 
        init_inv=init_inv, 
        trade_size=trade_size
    )

    random_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)

    return fixed_winning_percent, fixed_bar_winning_percent, random_winning_percent

def get_entry_win_pc_df(entry_walk_forward_dict, n_not_worked, n_total_cases):
    
    entry_mean_win_perc_dict = defaultdict(list)

    entry_mean_win_perc_dict['Fixed_StopLoss_TakeProfit_testing'].\
        append(np.mean(entry_walk_forward_dict['fixed_sp_testing']))
    entry_mean_win_perc_dict['Fixed_Bar_testing'].\
        append(np.mean(entry_walk_forward_dict['fixed_bar_testing']))
    entry_mean_win_perc_dict['Random_Exit_testing'].\
        append(np.mean(entry_walk_forward_dict['random_exit_testing']))
    entry_mean_win_perc_dict['Not_Working'].\
        append(100 * n_not_worked / n_total_cases)
        
    entry_win_pc_df = pd.DataFrame(entry_mean_win_perc_dict)

    return entry_win_pc_df

@njit(cache=True)
def get_entry_exit_testing1(
    close_prices,
    open_prices,
    n_bars
):

    signal_list = np.zeros((2, len(close_prices)))

    # n_bars = np.int64(n_bars)

    for i in range(n_bars, len(signal_list[0])):
            
        if close_prices[i - n_bars] - close_prices[i - 1] > 0:
            signal_list[0][i] = 1
            signal_list[1][i] = open_prices[i]
        elif close_prices[i - n_bars] - close_prices[i - 1] < 0:
           signal_list[0][i] = -1
           signal_list[1][i] = open_prices[i]
        else:
            pass
        
    return signal_list

@njit(cache=True)
def get_rsi(close_prices, prev_close_prices, length=14):
    # Create numpy arrays to store the gain/loss values
    gains = np.zeros(len(close_prices))
    losses = np.zeros(len(close_prices))

    # Iterate through the data frame and calculate the gain/loss for each period
    for i in range(1, len(close_prices)):
        change = close_prices[i] - prev_close_prices[i]
        if change > 0:
            gains[i] = change
        elif change < 0:
            losses[i] = abs(change)

    # Calculate the average gain and loss for each period
    avg_gains = np.zeros(len(close_prices))
    avg_losses = np.zeros(len(close_prices))
    for i in range(length, len(close_prices)):
        avg_gains[i] = np.mean(gains[i-length:i])
        avg_losses[i] = np.mean(losses[i-length:i])

    # Calculate the relative strength and RSI for each period
    rs = np.zeros(len(close_prices))
    rsi = np.zeros(len(close_prices))
    
    for i in range(len(close_prices)):
        if i+1 < length:
            rsi[i] = -999
        elif avg_losses[i] == 0:
            rs[i] = avg_gains[i]
            rsi[i] = 100
        else:
            rs[i] = avg_gains[i] / avg_losses[i]
            rsi[i] = 100 - (100 / (1 + rs[i]))

    return rsi

@njit(cache=True)
def get_entry_exit_testing2(
    close_prices, open_prices, prev_close_prices, rsi_window_size, rsi_threshold
):

    signal_list = np.zeros((2, len(close_prices)))

    rsi = get_rsi(close_prices, prev_close_prices, length=rsi_window_size)

    for i in range(len(close_prices)-1):

        if i < rsi_window_size - 1 or rsi[i] == -999:
            continue
       
        if rsi[i] < rsi_threshold:
            signal_list[0][i+1] = 1
            signal_list[1][i+1] = open_prices[i+1]
        elif rsi[i] > (100 - rsi_threshold):
            signal_list[0][i+1] = -1
            signal_list[1][i+1] = open_prices[i+1]
        else:
            pass

    return signal_list

@njit(cache=True)
def get_entry_exit_testing3(
    open_prices
):

    signal_list = np.zeros((2, len(open_prices)))

    for i in range(len(open_prices)-1):

        if np.random.rand() > 0.7:
            signal_list[0][i] = 1
            signal_list[1][i] = open_prices[i]
        elif np.random.rand() < 0.3:
            signal_list[0][i] = -1
            signal_list[1][i] = open_prices[i]
        else:
            pass

    return signal_list

def calculate_mean_win_perc_exit_testing(exec_dict, df):

    commission = exec_dict['COMMISSION']
    slippage = exec_dict['SLIPPAGE'] 
    init_inv = exec_dict['AVAILABLE_CAPITAL']
    trade_size = exec_dict['TRADE_SIZE']       

    if len(exec_dict['sell_exit_idxs']) != 0:
        signal_idxs = list(exec_dict['sell_exit_idxs'])
        if len(exec_dict['buy_exit_idxs']) != 0:
            signal_idxs.extend(list(exec_dict['buy_exit_idxs']))
    else:
        if len(exec_dict['buy_exit_idxs']) != 0:
            signal_idxs = list(exec_dict['buy_exit_idxs'])
        else:
            signal_idxs = []

    if len(signal_idxs) != 0:
        signal_idxs = sorted(signal_idxs)
        signal_idxs_true = [i - 1 for i in signal_idxs]
    else:
        signal_idxs_true = []

    df['buy'] = 0
    if len(exec_dict['sell_exit_idxs']) != 0:
        buy_idxs = [i - 1 for i in list(exec_dict['sell_exit_idxs'])]
        df.loc[df.index.isin(buy_idxs), 'buy'] = 1

    df['sell'] = 0
    if len(exec_dict['buy_exit_idxs']) != 0:
        sell_idxs = [i - 1 for i in list(exec_dict['buy_exit_idxs'])]
        df.loc[df.index.isin(sell_idxs), 'sell'] = -1

    df['signal'] = df['buy'] + df['sell']

    df['exit_signal'] = 0
    df['exit_prices'] = 0
    if len(signal_idxs) != 0:
        df.loc[df.index.isin(signal_idxs_true[1:]), 'exit_signal'] = df.loc[df.index.isin(signal_idxs_true[1:]), 'signal'].values
        df.loc[df.index.isin(signal_idxs[1:]), 'exit_prices'] = df.loc[df.index.isin(signal_idxs[1:]), 'btc_open'].values

    exit_list = np.zeros((2, df.shape[0]))
    exit_list[0][1:] = df['exit_signal'].values[:-1]
    exit_list[1] = df['exit_prices'].values

    # replacing entry with trend following entry
    signal_list = get_entry_exit_testing1(
        close_prices=df['btc_close'].values,
        open_prices=df['btc_open'].values,
        n_bars=5
    )

    signal_list, exit_list = get_signals(signal_list, exit_list)
    pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

    pnl_list = get_pnl_testing(
        trade_close_prices=pos_exit_prices,
        signal_list=exit_list[0], 
        trade_open_prices=pos_open_prices,
        commission=commission, 
        slippage=slippage, 
        init_inv=init_inv, 
        trade_size=trade_size
    )

    trend_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)

    # replacing entry with countertrend entry
    signal_list = get_entry_exit_testing2(
        close_prices=df['btc_close'].values, 
        open_prices=df['btc_open'].values, 
        prev_close_prices=df['btc_close'].shift(1).fillna(method='bfill').values, 
        rsi_window_size=10, 
        rsi_threshold = 20
    )

    signal_list, exit_list = get_signals(signal_list, exit_list)
    pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

    pnl_list = get_pnl_testing(
        trade_close_prices=pos_exit_prices,
        signal_list=exit_list[0], 
        trade_open_prices=pos_open_prices,
        commission=commission, 
        slippage=slippage, 
        init_inv=init_inv, 
        trade_size=trade_size
    )

    countertrend_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)

    # random entry testing
    signal_list = get_entry_exit_testing3(
        open_prices=df['btc_open'].values
    )

    signal_list, exit_list = get_signals(signal_list, exit_list)
    pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

    pnl_list = get_pnl_testing(
        trade_close_prices=pos_exit_prices,
        signal_list=exit_list[0], 
        trade_open_prices=pos_open_prices,
        commission=commission, 
        slippage=slippage, 
        init_inv=init_inv, 
        trade_size=trade_size
    )

    random_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)


    return trend_winning_percent, countertrend_winning_percent, random_winning_percent

def get_exit_win_pc_df(exit_walk_forward_dict, n_not_worked, n_total_cases):

    exit_mean_win_perc_dict = defaultdict(list)

    exit_mean_win_perc_dict['Trend_testing'].\
        append(np.mean(exit_walk_forward_dict['trend_entry_testing']))
    exit_mean_win_perc_dict['Countertrend_testing'].\
        append(np.mean(exit_walk_forward_dict['countertrend_entry_testing']))
    exit_mean_win_perc_dict['Random_Entry_testing'].\
        append(np.mean(exit_walk_forward_dict['random_entry_testing']))
    exit_mean_win_perc_dict['Not_Working'].\
        append(100 * n_not_worked / n_total_cases)

    exit_win_pc_df = pd.DataFrame(exit_mean_win_perc_dict)

    return exit_win_pc_df

def calculate_mean_win_perc_core_testing(exec_dict):
            
    pnl_list = exec_dict['all_arr']

    winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)
    
    return winning_percent

def get_core_win_pc_df(core_walk_forward_dict, n_not_worked, n_total_cases):

    core_mean_win_perc_dict = defaultdict(list)
    core_mean_win_perc_dict['Core_Testing'].\
        append(np.mean(core_walk_forward_dict['core_testing']))
    core_mean_win_perc_dict['Not_Working'].\
        append(100 * n_not_worked / n_total_cases)

    core_win_pc_df = pd.DataFrame(core_mean_win_perc_dict)

    return core_win_pc_df

@njit(cache=True)
def get_drawdown(pnl_list):
    max_dd = 0.0
    peak = pnl_list[0]

    for i in range(1, len(pnl_list)):
        if pnl_list[i] > peak:
            peak = pnl_list[i]
        drawdown = 100 * (peak - pnl_list[i]) / peak
        if drawdown > max_dd:
            max_dd = drawdown

    return max_dd

@njit(cache=True)
def get_drawdown_duration(pnl_arr):
    peak = pnl_arr[0]
    max_duration = 0
    current_duration = 0

    for pnl in pnl_arr:
        if pnl < peak:  # We are in a drawdown
            current_duration += 1
        else:  # We found a new peak
            peak = pnl
            current_duration = 0

        max_duration = max(max_duration, current_duration)

    return max_duration

@njit(cache=True)
def get_sharpe_ratio(
    pnl_list, 
    risk_free_rate=0
):
    # mean_return = np.mean(pnl_list)
    mean_return = 0
    for i in range(len(pnl_list)):
        mean_return += pnl_list[i]

    if len(pnl_list) == 0:
        mean_return = 0
    else:
        mean_return = mean_return / len(pnl_list)

    # std_dev = np.std(pnl_list)
    std_dev = 0
    for i in range(len(pnl_list)):
        std_dev += (pnl_list[i] - mean_return) ** 2

    if len(pnl_list) == 1:
        std_dev = np.sqrt(std_dev/len(pnl_list))
    else:
        std_dev = np.sqrt(std_dev/(len(pnl_list) - 1))

    if std_dev == 0:
        std_dev += 0.0001

    sharpe_ratio = (mean_return - risk_free_rate) / std_dev
    return sharpe_ratio

@njit(cache=True)
def get_sortino_ratio(profit_loss, target_pl=0):

    downside_pnl = profit_loss[profit_loss < target_pl]
    if len(downside_pnl) == 0:
        return 0

    downside_deviation = np.sqrt(np.mean((downside_pnl - target_pl) ** 2))
    if downside_deviation == 0:
        return 0

    mean_pnl = np.mean(profit_loss)
    sortino_ratio = (mean_pnl - target_pl) / downside_deviation

    return sortino_ratio

def calculate_mean_performance(exec_dict, monkey_test=False):

    pnl_list = exec_dict['all_arr']

    init_inv = exec_dict['AVAILABLE_CAPITAL']
    trade_size = exec_dict['TRADE_SIZE']       

    signal_idxs = list(exec_dict['buy_idxs'])
    signal_idxs.extend(list(exec_dict['sell_idxs']))
    signal_idxs = sorted(signal_idxs)

    metric_dict = {}

    metric_dict['n_trades'] = len(signal_idxs) - 1
    metric_dict['overall_pnl'] = np.sum(pnl_list)
    metric_dict['roi'] = 100 * metric_dict['overall_pnl'] / (trade_size * init_inv)
    metric_dict['avg_drawdown'] = exec_dict['avg_drawdown']
    metric_dict['max_dd'] = get_drawdown(pnl_list)
    metric_dict['drawdown_dur'] = get_drawdown_duration(pnl_list)
    metric_dict['pnl_avgd_ratio'] = exec_dict['fitness']

    annualized_sharpe_ration = np.sqrt(525600) * get_sharpe_ratio(pnl_list, risk_free_rate=0)
    metric_dict['sharpe_ratio'] = annualized_sharpe_ration

    annualized_sortino_ratio = np.sqrt(525600) * get_sortino_ratio(pnl_list)
    metric_dict['sortino_ratio'] = annualized_sortino_ratio

    if monkey_test:

        pnl_mren_arr = exec_dict['pnl_mren_arr']
        max_dd_mren_arr = exec_dict['max_dd_mren_arr']

        pnl_mren_good_cases = np.sum(np.where(metric_dict['overall_pnl'] > pnl_mren_arr, 1, 0))
        metric_dict['mt_pnl'] = 100 * pnl_mren_good_cases / len(pnl_mren_arr)

        max_dd_mren_good_cases = np.sum(np.where(metric_dict['max_dd'] < max_dd_mren_arr, 1, 0))
        metric_dict['mt_mdd'] = 100 * max_dd_mren_good_cases / len(max_dd_mren_arr)

    return metric_dict

def get_perf_df(performance_walk_forward_dict, n_not_worked, n_total_cases):

    mean_perf_dict = defaultdict(list)
    
    mean_perf_dict['N_Trades'].\
        append(np.nanmean(performance_walk_forward_dict['n_trades']))
    mean_perf_dict['PNL'].\
        append(np.nanmean(performance_walk_forward_dict['pnl']))
    mean_perf_dict['ROI (%)'].\
        append(np.nanmean(performance_walk_forward_dict['roi']))
    mean_perf_dict['AVG_Drawdown'].\
        append(np.nanmean(performance_walk_forward_dict['avg_drawdown']))
    mean_perf_dict['Drawdown (%)'].\
        append(np.nanmean(performance_walk_forward_dict['drawdown']))
    mean_perf_dict['Drawdown_Duration'].\
        append(np.nanmean(performance_walk_forward_dict['drawdown_dur']))
    mean_perf_dict['PNL_AVGD_Ratio'].\
        append(np.nanmean(performance_walk_forward_dict['pnl_avgd_ratio']))
    mean_perf_dict['Sharpe_Ratio'].\
        append(np.nanmean(performance_walk_forward_dict['sharpe_ratio']))
    mean_perf_dict['Sortino_Ratio'].\
        append(np.nanmean(performance_walk_forward_dict['sortino_ratio']))
    mean_perf_dict['Not_Working'].\
        append(100 * n_not_worked / n_total_cases)
    
    if 'mt_pnl' in performance_walk_forward_dict.keys():
        mean_perf_dict['MT_PNL_D'].\
            append(np.nanmean(performance_walk_forward_dict['mt_pnl']))
        mean_perf_dict['MT_MDD_D'].\
            append(np.nanmean(performance_walk_forward_dict['mt_mdd']))

    perf_df = pd.DataFrame(mean_perf_dict)

    return perf_df

@njit(cache=True)
def get_random_idxs4mc(arr, num_elements):
    # Get the length of the input array
    n = len(arr)

    if num_elements > n:
        raise ValueError("num_elements must be less than or equal to the length of the array.")

    # Generate random indices with replacement
    indices = np.random.randint(0, n, num_elements)

    # Return sorted indices
    return indices

@njit(cache=True)
def run_simulation(pnl_list, n_runs, init_inv, trade_size):

    max_dd_array = np.zeros(n_runs)
    dd_dur_array = np.zeros(n_runs)
    profit_array = np.zeros(n_runs)
    roi_array = np.zeros(n_runs)
    binary_profit_array = np.zeros(n_runs, dtype=np.int32)
    
    for i in range(n_runs):
        
        # Generate bootstrap sample
        temp_random_idxs = get_random_idxs4mc(arr=pnl_list, num_elements=len(pnl_list))
        temp_pnl_list = np.zeros(len(temp_random_idxs))
        for j in range(len(temp_random_idxs)):
            temp_pnl_list[j] = pnl_list[temp_random_idxs[j]]

        equity_curve_list = np.cumsum(temp_pnl_list)
        
        # Max Drawdown
        max_dd = get_drawdown(equity_curve_list)
        max_dd_array[i] = max_dd
        
        # Drawdown Duration
        dd_dur = get_drawdown_duration(equity_curve_list)
        dd_dur_array[i] = dd_dur
        
        # Profit
        profit = np.sum(temp_pnl_list)
        profit_array[i] = profit
        
        # ROI
        roi = 100 * profit / (init_inv * trade_size)
        roi_array[i] = roi
        
        # Binary Profit
        if profit > 0:
            binary_profit_array[i] = 1

    return max_dd_array, dd_dur_array, profit_array, roi_array, binary_profit_array

def get_mc_results(pnl_list, init_inv, trade_size, n_runs):

    max_dd_array, dd_dur_array, profit_array, roi_array, binary_profit_array = run_simulation(pnl_list, n_runs, init_inv, trade_size)

    mc_dict = {}
    mc_dict['median_max_dd'] = np.median(max_dd_array)
    mc_dict['median_dd_dur'] = np.median(dd_dur_array)
    mc_dict['median_profit'] = np.median(profit_array)
    mc_dict['median_return'] = np.median(roi_array)
    mc_dict['return_dd_ratio'] = np.median(roi_array) / np.median(max_dd_array)
    mc_dict['prob_profit'] = np.sum(binary_profit_array) / len(binary_profit_array)

    return mc_dict

def calculate_mc_performance(exec_dict):
            
    pnl_list = exec_dict['all_arr']

    init_inv = exec_dict['AVAILABLE_CAPITAL']
    trade_size = exec_dict['TRADE_SIZE']       

    mc_dict = get_mc_results(pnl_list, init_inv, trade_size, n_runs=10000)

    return mc_dict

def get_mc_df(mc_walk_forward_dict, n_not_worked, n_total_cases):

    mean_mc_dict = defaultdict(list)

    mean_mc_dict['median_drawdown (%)'].\
        append(np.mean(mc_walk_forward_dict['median_max_dd']))
    mean_mc_dict['median_drawdown_duration'].\
        append(np.mean(mc_walk_forward_dict['median_dd_dur']))
    mean_mc_dict['median_profit'].\
        append(np.mean(mc_walk_forward_dict['median_profit']))
    mean_mc_dict['median_ROI (%)'].\
        append(np.mean(mc_walk_forward_dict['median_return']))
    mean_mc_dict['ratio'].\
        append(np.mean(mc_walk_forward_dict['return_dd_ratio']))
    mean_mc_dict['prob'].\
        append(np.mean(mc_walk_forward_dict['prob_profit']))
    mean_mc_dict['Not_Working'].\
        append(100 * n_not_worked / n_total_cases)

    mc_df = pd.DataFrame(mean_mc_dict)

    return mc_df



# Loading results for btc_experiment from 2023-11-01 18:56:00 to 2024-09-06 20:54:00

In [2]:
final_entry_win_pc_df = pd.read_csv('testing_results/entry_testing_btc_experiment.csv')
final_exit_win_pc_df = pd.read_csv('testing_results/exit_testing_btc_experiment.csv')
final_core_win_pc_df = pd.read_csv('testing_results/core_testing_btc_experiment.csv')
final_perf_df = pd.read_csv('testing_results/perf_btc_experiment.csv')
final_mc_df = pd.read_csv('testing_results/mc_btc_experiment.csv')

In [5]:
# from pathlib import Path

# data_path = Path(r'C:/\Users/\vchar/\OneDrive/\Desktop/\ML Projects/\Upwork/\AlgoT_ML_Dev/\GrammarEvolution/\PonyGE2/\datasets/\all_data_1min.csv')
# temp_df = pd.read_csv(data_path)
# temp_df['datetime'] = pd.to_datetime(temp_df['datetime'])
# temp_df.sort_values('datetime', ascending=True, inplace=True)
# temp_df.reset_index(inplace=True, drop=True)
# temp_df

In [16]:
strategy_file_path = Path(r"C:/\Users/\vchar/\OneDrive/\Desktop/\ML Projects/\Upwork/\AlgoT_ML_Dev/\GrammarEvolution/\PonyGE2/\results/\VCh_24_10_30_180400_225305_2676_225305/\ge_results.csv")

try:
    df_str = pd.read_csv(strategy_file_path)
except:
    df_str = pd.read_csv(strategy_file_path, sep=';')
    
# df_str = df_str[(df_str['fitness'] < 0) & (df_str['fitness'] < -3) & (df_str['fitness'] > -50)]
df_str = df_str[(df_str['fitness'] < 0) & (df_str['fitness'] <= -1.5)]
df_str = df_str[~df_str.duplicated()]
df_str.sort_values('fitness', ascending=True, inplace=True)
df_str.reset_index(drop=True, inplace=True)
df_str['strategy'] = list(map(lambda x: f"strategy{x}", np.arange(1, df_str.shape[0]+1)))
df_str.head()

,buy,sell,fitness,strategy
0,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,-61876.559872,strategy1
1,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,-61876.559872,strategy2
2,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,-61876.559872,strategy3
3,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,-61876.559872,strategy4
4,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,-61876.559872,strategy5


# Analysing results for btc_experiment from 2023-11-01 18:56:00 to 2024-09-06 20:54:00

In [11]:
# finding strategies that have winning percentage equal to 60 or more for entry testing cases
entry_testing_strategies = final_entry_win_pc_df[
    (final_entry_win_pc_df['Fixed_StopLoss_TakeProfit_testing'] >= 60) & 
    (final_entry_win_pc_df['Fixed_Bar_testing'] >= 60) & 
    (final_entry_win_pc_df['Random_Exit_testing'] >= 60)
]['strategy'].tolist()

# finding strategies that have passed entry testing and have winning percentage equal to 60 
# or more (except for Random_Entry_testing) for exit testing cases
exit_testing_strategies = final_exit_win_pc_df[
    (final_exit_win_pc_df['Trend_testing'] >= 60) & 
    (final_exit_win_pc_df['Countertrend_testing'] >= 60) & 
    (final_exit_win_pc_df['Random_Entry_testing'] >= 50) &
    (final_exit_win_pc_df['strategy'].isin(entry_testing_strategies))
]['strategy'].tolist()

# finding strategies that has passed both entry and exit testing and 
# have winning percentage equal to 60 or more for core testing cases
core_testing_strategies = final_core_win_pc_df[
    (final_core_win_pc_df['Core_Testing'] >= 60) &
    (final_core_win_pc_df['strategy'].isin(exit_testing_strategies))
]['strategy'].tolist()

# finding performance for strategies that has passed all tests and have positive ROI
perf_testing_strategies1 = final_perf_df[
    (final_perf_df['strategy'].isin(core_testing_strategies)) & 
    (final_perf_df['ROI (%)'] > 0)
]['strategy'].tolist()

# MT_PNL_D	MT_MDD_D
# finding performance for strategies that has passed all tests, have positive ROI
# and in 90 and more % of cases have better pnl when monkey testing is conducted
perf_testing_strategies2 = final_perf_df[
    (final_perf_df['strategy'].isin(core_testing_strategies)) & 
    (final_perf_df['ROI (%)'] > 0) & 
    (final_perf_df['MT_PNL_D'] > 90)
]['strategy'].tolist()

len(perf_testing_strategies1), len(perf_testing_strategies2)

(34, 4)

In [12]:
# Showing Monte Carlo simulation results for strategies that has passed entry, exit, and core testing and have positive ROI
final_mc_df[final_mc_df['strategy'].isin(perf_testing_strategies1)]

,strategy,buy,sell,median_drawdown (%),median_drawdown_duration,median_profit,median_ROI (%),ratio,prob,Not_Working
8446,strategy8447,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] > get_lag(...,10.181588,10.0,1.384047e+06,395.441961,38.838926,1.0000,0.0
9223,strategy9224,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] > numba_in...,1.662558,3.0,3.541272e+06,1011.791950,608.575545,1.0000,0.0
9463,strategy9464,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] > numba_in...,3.261562,3.0,4.219482e+06,1205.566150,369.628481,1.0000,0.0
12323,strategy12324,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] > numba_in...,8.760395,5.0,1.529723e+06,437.063616,49.890855,1.0000,0.0
13271,strategy13272,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] < get_lag(...,6.492154,5.0,1.544262e+06,441.217664,67.961674,1.0000,0.0
16439,strategy16440,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] == signals...,3.736622,3.0,1.303886e+06,372.538954,99.699390,1.0000,0.0
21214,strategy21215,((price_data['btc_close'][MAX_LAG:] > numba_in...,((price_data['btc_close'][MAX_LAG:] > numba_in...,4.557618,2.0,1.302347e+06,372.099093,81.643326,1.0000,0.0
24165,strategy24166,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] < get_lag(...,12.456906,7.0,9.806121e+05,280.174883,22.491530,1.0000,0.0
24535,strategy24536,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] > get_lag(...,21.216313,11.0,1.304646e+06,372.755952,17.569309,1.0000,0.0
26244,strategy26245,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] == signals...,9.505144,4.0,2.406440e+06,687.554175,72.334954,1.0000,0.0


In [13]:
# Showing Monte Carlo simulation results for strategies that has passed entry, exit, and core testing, 
# have positive ROI, and have better pnl compared to monkey strategies in 90 and % of cases
final_mc_df[final_mc_df['strategy'].isin(perf_testing_strategies2)]

,strategy,buy,sell,median_drawdown (%),median_drawdown_duration,median_profit,median_ROI (%),ratio,prob,Not_Working
34429,strategy34430,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,2.067886,1.0,2.986624e+06,853.321258,412.653898,1.0,0.0
37127,strategy37128,((price_data['btc_close'][MAX_LAG:] < get_lag(...,((numba_indicators.relative_strength_index(pri...,6.315974,1.0,1.291890e+06,369.111299,58.440911,1.0,0.0
37496,strategy37497,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,4.459222,1.0,1.231988e+06,351.996445,78.936736,1.0,0.0
37517,strategy37518,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,4.992538,1.0,1.094344e+06,312.669676,62.627396,1.0,0.0


In [ ]:
# Showing fitness for strategies that has passed entry, exit, and core testing and have positive ROI
df_str[df_str['strategy'].isin(perf_testing_strategies1)]

,buy,sell,fitness,strategy
8446,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] > get_lag(...,-955.736104,strategy8447
9223,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] > numba_in...,-920.225315,strategy9224
9463,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] > numba_in...,-910.409162,strategy9464
12323,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] > numba_in...,-797.149624,strategy12324
13271,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] < get_lag(...,-762.315127,strategy13272
16439,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] == signals...,-660.897307,strategy16440
21214,((price_data['btc_close'][MAX_LAG:] > numba_in...,((price_data['btc_close'][MAX_LAG:] > numba_in...,-487.745542,strategy21215
24165,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] < get_lag(...,-372.215537,strategy24166
24535,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] > get_lag(...,-358.782633,strategy24536
26244,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] == signals...,-284.615847,strategy26245


In [ ]:
# Showing fitness for strategies that has passed entry, exit, and core testing, 
# have positive ROI, and have better pnl compared to monkey strategies in 90 and % of cases
df_str[df_str['strategy'].isin(perf_testing_strategies2)]

,buy,sell,fitness,strategy
34429,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,-40.983778,strategy34430
37127,((price_data['btc_close'][MAX_LAG:] < get_lag(...,((numba_indicators.relative_strength_index(pri...,-6.024938,strategy37128
37496,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,-4.193188,strategy37497
37517,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,-4.126115,strategy37518


In [24]:
for strategy in perf_testing_strategies2:
    print('#' * 10 + f'Strategy {perf_testing_strategies2.index(strategy)+1}' + '#' * 10)
    print('Buy:', df_str[df_str['strategy']==strategy]['buy'].values.tolist()[0])
    print('Sell:', df_str[df_str['strategy']==strategy]['sell'].values.tolist()[0])

##########Strategy 1##########
Buy: ((price_data['btc_close'][MAX_LAG:] < numba_indicators.moving_average(prices=price_data['btc_close'], window=86)[MAX_LAG:]) & (price_data['btc_close'][MAX_LAG:] > numba_indicators.moving_average(prices=price_data['btc_close'], window=432)[MAX_LAG:]))
Sell: ((numba_indicators.relative_strength_index(prices=price_data['btc_close'], window=37)[MAX_LAG:] < 30) & (numba_indicators.relative_strength_index(prices=price_data['btc_close'], window=771)[MAX_LAG:] > 30))
##########Strategy 2##########
Buy: ((price_data['btc_close'][MAX_LAG:] < get_lag(price_data['btc_close'], lag=59)[MAX_LAG:]) & (price_data['btc_close'][MAX_LAG:] > numba_indicators.moving_average(prices=price_data['btc_close'], window=837)[MAX_LAG:]))
Sell: ((numba_indicators.relative_strength_index(prices=price_data['btc_close'], window=69)[MAX_LAG:] < 30) & (price_data['btc_close'][MAX_LAG:] < get_lag(price_data['btc_close'], lag=70)[MAX_LAG:]))
##########Strategy 3##########
Buy: ((price_dat

# Loading results for btc_test from 2024-02-14 18:14:00 to 2024-03-20 18:13:00

In [32]:
final_entry_win_pc_df = pd.read_csv('testing_results/entry_testing_btc_test.csv')
final_exit_win_pc_df = pd.read_csv('testing_results/exit_testing_btc_test.csv')
final_core_win_pc_df = pd.read_csv('testing_results/core_testing_btc_test.csv')
final_perf_df = pd.read_csv('testing_results/perf_btc_test.csv')
final_mc_df = pd.read_csv('testing_results/mc_btc_test.csv')

In [33]:
strategy_file_path = Path(r'C:/\Users/\vchar/\OneDrive/\Desktop/\ML Projects/\Upwork/\AlgoT_ML_Dev/\GrammarEvolution/\PonyGE2/\results/\VCh_24_11_1_120353_214176_28464_214176/\ge_results.csv')

try:
    df_str = pd.read_csv(strategy_file_path)
except:
    df_str = pd.read_csv(strategy_file_path, sep=';')
    
# df_str = df_str[(df_str['fitness'] < 0) & (df_str['fitness'] < -3) & (df_str['fitness'] > -50)]
df_str = df_str[(df_str['fitness'] < 0) & (df_str['fitness'] <= -1.5)]
df_str = df_str[~df_str.duplicated()]
df_str.sort_values('fitness', ascending=True, inplace=True)
df_str.reset_index(drop=True, inplace=True)
df_str['strategy'] = list(map(lambda x: f"strategy{x}", np.arange(1, df_str.shape[0]+1)))
df_str.head()

,buy,exit_buy,sell,exit_sell,fitness,strategy
0,(numba_indicators.moving_average(numba_indicat...,(price_data['btc_close'][MAX_LAG:] < signals.m...,"((get_lag(price_data['btc_low'], lag=2)[MAX_LA...",(price_data['btc_close'][MAX_LAG:] < signals.m...,-1147.204153,strategy1
1,(numba_indicators.moving_average(numba_indicat...,(price_data['btc_close'][MAX_LAG:] < signals.m...,"((get_lag(price_data['btc_low'], lag=2)[MAX_LA...",(price_data['btc_close'][MAX_LAG:] < signals.m...,-1127.612955,strategy2
2,(numba_indicators.moving_average(numba_indicat...,(price_data['btc_close'][MAX_LAG:] < signals.m...,"((get_lag(price_data['btc_low'], lag=2)[MAX_LA...",(price_data['btc_close'][MAX_LAG:] < signals.m...,-1111.725509,strategy3
3,(numba_indicators.moving_average(numba_indicat...,(price_data['btc_close'][MAX_LAG:] < signals.m...,"((get_lag(price_data['btc_low'], lag=2)[MAX_LA...",(price_data['btc_close'][MAX_LAG:] < signals.m...,-1099.944450,strategy4
4,(numba_indicators.moving_average(numba_indicat...,(price_data['btc_close'][MAX_LAG:] < signals.m...,"((get_lag(price_data['btc_low'], lag=2)[MAX_LA...",(price_data['btc_close'][MAX_LAG:] < signals.m...,-1096.639815,strategy5


# Analysing results for btc_test from 2024-02-14 18:14:00 to 2024-03-20 18:13:00

In [36]:
# finding strategies that have winning percentage equal to 60 or more for entry testing cases
entry_testing_strategies = final_entry_win_pc_df[
    (final_entry_win_pc_df['Fixed_StopLoss_TakeProfit_testing'] >= 60) & 
    (final_entry_win_pc_df['Fixed_Bar_testing'] >= 60) & 
    (final_entry_win_pc_df['Random_Exit_testing'] >= 60)
]['strategy'].tolist()

# finding strategies that have passed entry testing and have winning percentage equal to 60 
# or more (except for Random_Entry_testing) for exit testing cases
exit_testing_strategies = final_exit_win_pc_df[
    (final_exit_win_pc_df['Trend_testing'] >= 60) & 
    (final_exit_win_pc_df['Countertrend_testing'] >= 60) & 
    (final_exit_win_pc_df['Random_Entry_testing'] >= 40) &
    (final_exit_win_pc_df['strategy'].isin(entry_testing_strategies))
]['strategy'].tolist()

# finding strategies that has passed both entry and exit testing and 
# have winning percentage equal to 60 or more for core testing cases
core_testing_strategies = final_core_win_pc_df[
    (final_core_win_pc_df['Core_Testing'] >= 60) &
    (final_core_win_pc_df['strategy'].isin(exit_testing_strategies))
]['strategy'].tolist()

# finding performance for strategies that has passed all tests and have positive ROI
perf_testing_strategies1 = final_perf_df[
    (final_perf_df['strategy'].isin(core_testing_strategies)) & 
    (final_perf_df['ROI (%)'] > 0)
]['strategy'].tolist()

# MT_PNL_D	MT_MDD_D
# finding performance for strategies that has passed all tests, have positive ROI
# and in 90 and more % of cases have better pnl when monkey testing is conducted
perf_testing_strategies2 = final_perf_df[
    (final_perf_df['strategy'].isin(core_testing_strategies)) & 
    (final_perf_df['ROI (%)'] > 0)
]['strategy'].tolist()

len(perf_testing_strategies1), len(perf_testing_strategies2)

(29, 29)

In [37]:
# Showing fitness for strategies that has passed entry, exit, and core testing and have positive ROI
df_str[df_str['strategy'].isin(perf_testing_strategies1)]

,buy,exit_buy,sell,exit_sell,fitness,strategy
4525,(numba_indicators.moving_average(numba_indicat...,(price_data['btc_close'][MAX_LAG:] < signals.m...,((numba_indicators.exponential_moving_average(...,(price_data['btc_close'][MAX_LAG:] < signals.m...,-276.864043,strategy4526
8969,(numba_indicators.moving_average(numba_indicat...,(price_data['btc_close'][MAX_LAG:] < signals.m...,((numba_indicators.moving_average(prices=price...,(price_data['btc_close'][MAX_LAG:] < signals.m...,-111.012865,strategy8970
9167,(numba_indicators.moving_average(numba_indicat...,(price_data['day_of_week'][MAX_LAG:] >= 7),((numba_indicators.adx(high=price_data['btc_hi...,(price_data['day_of_week'][MAX_LAG:] >= 7),-101.677014,strategy9168
9190,((numba_indicators.moving_average(prices=price...,(price_data['day_of_week'][MAX_LAG:] >= 6),(numba_indicators.moving_average(numba_indicat...,(price_data['day_of_week'][MAX_LAG:] >= 6),-100.330915,strategy9191
9796,"((get_lag(price_data['btc_close'], lag=1)[MAX_...",(price_data['btc_close'][MAX_LAG:] < signals.m...,(numba_indicators.ultimate_oscillator(high=pri...,(price_data['btc_close'][MAX_LAG:] < signals.m...,-76.183653,strategy9797
9809,((price_data['day_of_week'][MAX_LAG:] == 5) & ...,(price_data['btc_close'][MAX_LAG:] < signals.m...,((numba_indicators.adx(high=price_data['btc_hi...,(price_data['btc_close'][MAX_LAG:] < signals.m...,-75.685306,strategy9810
10118,((numba_indicators.moving_average(prices=price...,(price_data['btc_close'][MAX_LAG:] < signals.m...,(price_data['day_of_week'][MAX_LAG:] <= 2),(price_data['btc_close'][MAX_LAG:] < signals.m...,-64.011114,strategy10119
10126,"((get_lag(price_data['btc_high'], lag=2)[MAX_L...",(price_data['btc_close'][MAX_LAG:] < signals.m...,((numba_indicators.exponential_moving_average(...,(price_data['btc_close'][MAX_LAG:] < signals.m...,-63.849167,strategy10127
10234,((numba_indicators.adx(high=price_data['btc_hi...,(price_data['btc_close'][MAX_LAG:] == signals....,((numba_indicators.moving_average(prices=price...,(price_data['btc_close'][MAX_LAG:] == signals....,-60.701312,strategy10235
10244,((numba_indicators.adx(high=price_data['btc_hi...,(price_data['day_of_week'][MAX_LAG:] <= 4),((numba_indicators.adx(high=price_data['btc_hi...,(price_data['day_of_week'][MAX_LAG:] <= 4),-60.266236,strategy10245


In [38]:
for strategy in perf_testing_strategies2:
    print('#' * 10 + f'Strategy {perf_testing_strategies2.index(strategy)+1}' + '#' * 10)
    print('Buy:', df_str[df_str['strategy']==strategy]['buy'].values.tolist()[0])
    print('Exit Buy:', df_str[df_str['strategy']==strategy]['exit_buy'].values.tolist()[0])
    print('Sell:', df_str[df_str['strategy']==strategy]['sell'].values.tolist()[0])
    print('Exit Sell:', df_str[df_str['strategy']==strategy]['exit_sell'].values.tolist()[0])

##########Strategy 1##########
Buy: (numba_indicators.moving_average(numba_indicators.commodity_channel_index(price_data['btc_high'], price_data['btc_low'], price_data['btc_close'], period=3), window=2)[MAX_LAG:] <= -100)
Exit Buy: (price_data['btc_close'][MAX_LAG:] < signals.moving_percentile(price_data['btc_close'], window=603, percentile=0.8)[MAX_LAG:])
Sell: ((numba_indicators.exponential_moving_average(prices=price_data['btc_close'], window=4)[MAX_LAG:] < numba_indicators.exponential_moving_average(prices=price_data['btc_close'], window=255)[MAX_LAG:]) & (get_lag(numba_indicators.exponential_moving_average(prices=price_data['btc_close'], window=8), lag=1)[MAX_LAG:] > get_lag(numba_indicators.exponential_moving_average(prices=price_data['btc_close'], window=4635), lag=1)[MAX_LAG:]))
Exit Sell: (price_data['btc_close'][MAX_LAG:] < signals.moving_percentile(price_data['btc_close'], window=603, percentile=0.8)[MAX_LAG:])
##########Strategy 2##########
Buy: (numba_indicators.moving_ave

In [46]:
path1 = Path(r"C:/\Users/\vchar/\Downloads/\ge_results (3).csv")
path2 = Path(r"C:/\Users/\vchar/\Downloads/\ge_results (4).csv")

df_str1 = pd.read_csv(path1, sep=';')
df_str1.sort_values('fitness', ascending=True, inplace=True)
df_str1.reset_index(drop=True, inplace=True)

df_str2 = pd.read_csv(path2, sep=';')
df_str2.sort_values('fitness', ascending=True, inplace=True)
df_str2.reset_index(drop=True, inplace=True)

In [49]:
df_str1.head()

,buy,exit_buy,sell,exit_sell,fitness
0,((numba_indicators.moving_average(prices=price...,(price_data['btc_close'][MAX_LAG:] < signals.m...,((numba_indicators.exponential_moving_average(...,(price_data['btc_close'][MAX_LAG:] < signals.m...,-1.760070e+06
1,((numba_indicators.moving_average(prices=price...,(price_data['btc_close'][MAX_LAG:] < signals.m...,((numba_indicators.exponential_moving_average(...,(price_data['btc_close'][MAX_LAG:] < signals.m...,-1.760070e+06
2,((numba_indicators.moving_average(prices=price...,(price_data['btc_close'][MAX_LAG:] < signals.m...,((numba_indicators.exponential_moving_average(...,(price_data['btc_close'][MAX_LAG:] < signals.m...,-1.752800e+06
3,((numba_indicators.moving_average(prices=price...,(price_data['btc_close'][MAX_LAG:] < signals.m...,((numba_indicators.exponential_moving_average(...,(price_data['btc_close'][MAX_LAG:] < signals.m...,-1.752800e+06
4,((numba_indicators.moving_average(prices=price...,(price_data['btc_close'][MAX_LAG:] < signals.m...,((numba_indicators.exponential_moving_average(...,(price_data['btc_close'][MAX_LAG:] < signals.m...,-1.751495e+06


In [50]:
df_str2.head()

,buy,exit_buy,sell,exit_sell,fitness
0,((price_data['day_of_week'][MAX_LAG:] == 5) & ...,(price_data['btc_close'][MAX_LAG:] < signals.m...,((numba_indicators.moving_average(prices=price...,(price_data['btc_close'][MAX_LAG:] < signals.m...,-46754.983977
1,((price_data['day_of_week'][MAX_LAG:] == 5) & ...,(price_data['btc_close'][MAX_LAG:] < signals.m...,((numba_indicators.moving_average(prices=price...,(price_data['btc_close'][MAX_LAG:] < signals.m...,-46754.983977
2,((price_data['day_of_week'][MAX_LAG:] == 5) & ...,(price_data['btc_close'][MAX_LAG:] < signals.m...,((numba_indicators.moving_average(prices=price...,(price_data['btc_close'][MAX_LAG:] < signals.m...,-46754.983977
3,((price_data['day_of_week'][MAX_LAG:] == 5) & ...,(price_data['btc_close'][MAX_LAG:] < signals.m...,((numba_indicators.moving_average(prices=price...,(price_data['btc_close'][MAX_LAG:] < signals.m...,-46754.983977
4,((price_data['day_of_week'][MAX_LAG:] == 5) & ...,(price_data['btc_close'][MAX_LAG:] < signals.m...,((numba_indicators.moving_average(prices=price...,(price_data['btc_close'][MAX_LAG:] < signals.m...,-46752.861702


In [47]:
print('Buy:', df_str1.iloc[0]['buy'])
print('Exit Buy:', df_str1.iloc[0]['exit_buy'])
print('Sell:', df_str1.iloc[0]['sell'])
print('Exit Sell:', df_str1.iloc[0]['exit_sell'])

Buy: ((numba_indicators.moving_average(prices=price_data['btc_close'], window=60)[MAX_LAG:] > numba_indicators.moving_average(prices=price_data['btc_close'], window=630)[MAX_LAG:]) & (price_data['btc_close'][MAX_LAG:] < (price_data['btc_low'][MAX_LAG:] + 2428)) & (get_lag(numba_indicators.moving_average(prices=price_data['btc_close'], window=1), lag=1)[MAX_LAG:] < get_lag(numba_indicators.moving_average(prices=price_data['btc_close'], window=9087), lag=1)[MAX_LAG:]))
Exit Buy: (price_data['btc_close'][MAX_LAG:] < signals.moving_percentile(price_data['btc_close'], window=1305, percentile=0.6)[MAX_LAG:])
Sell: ((numba_indicators.exponential_moving_average(prices=price_data['btc_close'], window=33)[MAX_LAG:] < numba_indicators.exponential_moving_average(prices=price_data['btc_close'], window=1771)[MAX_LAG:]) & (get_lag(numba_indicators.exponential_moving_average(prices=price_data['btc_close'], window=5), lag=1)[MAX_LAG:] > get_lag(numba_indicators.exponential_moving_average(prices=price_d

In [48]:
print('Buy:', df_str2.iloc[0]['buy'])
print('Exit Buy:', df_str2.iloc[0]['exit_buy'])
print('Sell:', df_str2.iloc[0]['sell'])
print('Exit Sell:', df_str2.iloc[0]['exit_sell'])

Buy: ((price_data['day_of_week'][MAX_LAG:] == 5) & (price_data['btc_close'][MAX_LAG:] == signals.moving_max(price_data['btc_close'], window=10)[MAX_LAG:]))
Exit Buy: (price_data['btc_close'][MAX_LAG:] < signals.moving_percentile(price_data['btc_close'], window=3203, percentile=0.32)[MAX_LAG:])
Sell: ((numba_indicators.moving_average(prices=price_data['btc_close'], window=72)[MAX_LAG:] < numba_indicators.moving_average(prices=price_data['btc_close'], window=2636)[MAX_LAG:]) & (price_data['btc_close'][MAX_LAG:] > (price_data['btc_high'][MAX_LAG:] - 7610)) & (get_lag(numba_indicators.moving_average(prices=price_data['btc_close'], window=4), lag=1)[MAX_LAG:] > get_lag(numba_indicators.moving_average(prices=price_data['btc_close'], window=246), lag=1)[MAX_LAG:]))
Exit Sell: (price_data['btc_close'][MAX_LAG:] < signals.moving_percentile(price_data['btc_close'], window=3203, percentile=0.32)[MAX_LAG:])


In [ ]:
((ma(60) > ma(630)) & (close < (low + 2428)) & (ma(1)[-1] < ma(9087)[-1]))
(close < signals.moving_percentile(price_data['btc_close'], window=1305, percentile=0.6)[MAX_LAG:])

((numba_indicators.exponential_moving_average(prices=price_data['btc_close'], window=33)[MAX_LAG:] < numba_indicators.exponential_moving_average(prices=price_data['btc_close'], window=1771)[MAX_LAG:]) & (get_lag(numba_indicators.exponential_moving_average(prices=price_data['btc_close'], window=5), lag=1)[MAX_LAG:] > get_lag(numba_indicators.exponential_moving_average(prices=price_data['btc_close'], window=9086), lag=1)[MAX_LAG:]))
Exit Sell: (price_data['btc_close'][MAX_LAG:] < signals.moving_percentile(price_data['btc_close'], window=1305, percentile=0.6)[MAX_LAG:])

In [55]:
data = pd.read_csv('data_5min.csv')
data = data.iloc[-50400:]
data.to_csv('data_5min_reduced.csv', index=False)

# Loading results for fold1: from 

In [19]:
final_entry_win_pc_df_fold1_p1 = pd.read_csv('testing_results/entry_testing_fold1_p1.csv')
final_exit_win_pc_df_fold1_p1 = pd.read_csv('testing_results/exit_testing_fold1_p1.csv')
final_core_win_pc_df_fold1_p1 = pd.read_csv('testing_results/core_testing_fold1_p1.csv')
final_perf_df_fold1_p1 = pd.read_csv('testing_results/perf_fold1_p1.csv')
final_mc_df_fold1_p1 = pd.read_csv('testing_results/mc_fold1_p1.csv')

final_entry_win_pc_df_fold1_p2 = pd.read_csv('testing_results/entry_testing_fold1_p2.csv')
final_exit_win_pc_df_fold1_p2 = pd.read_csv('testing_results/exit_testing_fold1_p2.csv')
final_core_win_pc_df_fold1_p2 = pd.read_csv('testing_results/core_testing_fold1_p2.csv')
final_perf_df_fold1_p2 = pd.read_csv('testing_results/perf_fold1_p2.csv')
final_mc_df_fold1_p2 = pd.read_csv('testing_results/mc_fold1_p2.csv')

strategy_file_path = Path(r"C:/\Users/\vchar/\Downloads/\ge_results (5).csv")

try:
    df_str_fold1_p1 = pd.read_csv(strategy_file_path)
except:
    df_str_fold1_p1 = pd.read_csv(strategy_file_path, sep=';')
    
df_str_fold1_p1 = df_str_fold1_p1[(df_str_fold1_p1['fitness'] < 0) & (df_str_fold1_p1['fitness'] <= -1000)]
df_str_fold1_p1 = df_str_fold1_p1[~df_str_fold1_p1.duplicated()]
df_str_fold1_p1.sort_values('fitness', ascending=True, inplace=True)
df_str_fold1_p1.reset_index(drop=True, inplace=True)
df_str_fold1_p1['strategy'] = list(map(lambda x: f"strategy{x}", np.arange(1, df_str_fold1_p1.shape[0]+1)))

strategy_file_path = Path(r"C:/\Users/\vchar/\Downloads/\ge_results (5).csv")

try:
    df_str_fold1_p2 = pd.read_csv(strategy_file_path)
except:
    df_str_fold1_p2 = pd.read_csv(strategy_file_path, sep=';')

df_str_fold1_p2 = df_str_fold1_p2[(df_str_fold1_p2['fitness'] < 0) & (df_str_fold1_p2['fitness'] > -1000)]
df_str_fold1_p2 = df_str_fold1_p2[~df_str_fold1_p2.duplicated()]
df_str_fold1_p2.sort_values('fitness', ascending=True, inplace=True)
df_str_fold1_p2.reset_index(drop=True, inplace=True)
df_str_fold1_p2 = df_str_fold1_p2.iloc[:5000]
df_str_fold1_p2['strategy'] = list(map(lambda x: f"strategy{x}", np.arange(1, df_str_fold1_p2.shape[0]+1)))

# Analysing results for fold1: from

### Part 1

In [20]:
# finding strategies that have winning percentage equal to 60 or more for entry testing cases
entry_testing_strategies = final_entry_win_pc_df_fold1_p1[
    (final_entry_win_pc_df_fold1_p1['Fixed_StopLoss_TakeProfit_testing'] >= 60) & 
    (final_entry_win_pc_df_fold1_p1['Fixed_Bar_testing'] >= 60) & 
    (final_entry_win_pc_df_fold1_p1['Random_Exit_testing'] >= 60)
]['strategy'].tolist()

# finding strategies that have passed entry testing and have winning percentage equal to 60 
# or more (except for Random_Entry_testing) for exit testing cases
exit_testing_strategies = final_exit_win_pc_df_fold1_p1[
    (final_exit_win_pc_df_fold1_p1['Trend_testing'] >= 60) & 
    (final_exit_win_pc_df_fold1_p1['Countertrend_testing'] >= 60) & 
    (final_exit_win_pc_df_fold1_p1['Random_Entry_testing'] >= 40) &
    (final_exit_win_pc_df_fold1_p1['strategy'].isin(entry_testing_strategies))
]['strategy'].tolist()

# finding strategies that have passed both entry and exit testing and 
# have winning percentage equal to 60 or more for core testing cases
core_testing_strategies1 = final_core_win_pc_df_fold1_p1[
    (final_core_win_pc_df_fold1_p1['Core_Testing'] >= 60) &
    (final_core_win_pc_df_fold1_p1['strategy'].isin(exit_testing_strategies))
]['strategy'].tolist()

# finding strategies that have passed entry testing and 
# have winning percentage equal to 60 or more for core testing cases
core_testing_strategies2 = final_core_win_pc_df_fold1_p1[
    (final_core_win_pc_df_fold1_p1['Core_Testing'] >= 60) &
    (final_core_win_pc_df_fold1_p1['strategy'].isin(entry_testing_strategies))
]['strategy'].tolist()

# finding performance for strategies that have passed all tests and have positive ROI
perf_testing_strategies1 = final_perf_df_fold1_p1[
    (final_perf_df_fold1_p1['strategy'].isin(core_testing_strategies1)) & 
    (final_perf_df_fold1_p1['ROI (%)'] > 0) & 
    (final_perf_df_fold1_p1['Drawdown (%)'] < 100)
]['strategy'].tolist()

# finding performance for strategies that have passed entry testing, core testing, and have positive ROI
perf_testing_strategies2 = final_perf_df_fold1_p1[
    (final_perf_df_fold1_p1['strategy'].isin(core_testing_strategies2)) & 
    (final_perf_df_fold1_p1['ROI (%)'] > 0) & 
    (final_perf_df_fold1_p1['Drawdown (%)'] < 100)
]['strategy'].tolist()

len(perf_testing_strategies1), len(perf_testing_strategies2)

(4, 4)

In [21]:
df_str_fold1_p1 = df_str_fold1_p1[df_str_fold1_p1['strategy'].isin(perf_testing_strategies2)]
df_str_fold1_p1.drop(columns='strategy', inplace=True)
df_str_fold1_p1

,buy,exit_buy,sell,exit_sell,fitness
4604,(price_data['googl_close'][MAX_LAG:] > get_lag...,"(get_lag(price_data['aapl_close'], lag=8)[MAX_...",((numba_indicators.moving_average(prices=price...,"(get_lag(price_data['aapl_close'], lag=8)[MAX_...",-1722.437713
4658,(price_data['googl_close'][MAX_LAG:] > get_lag...,"(get_lag(price_data['aapl_close'], lag=8)[MAX_...",((numba_indicators.moving_average(prices=price...,"(get_lag(price_data['aapl_close'], lag=8)[MAX_...",-1631.689868
4797,((numba_indicators.moving_average(prices=price...,(price_data['day_of_week'][MAX_LAG:] >= 4),(numba_indicators.ultimate_oscillator(high=pri...,(price_data['day_of_week'][MAX_LAG:] >= 4),-1471.767035
5146,((numba_indicators.moving_average(prices=price...,(price_data['day_of_week'][MAX_LAG:] >= 4),(numba_indicators.ultimate_oscillator(high=pri...,(price_data['day_of_week'][MAX_LAG:] >= 4),-1146.334724


### Part2

In [22]:
# finding strategies that have winning percentage equal to 60 or more for entry testing cases
entry_testing_strategies = final_entry_win_pc_df_fold1_p2[
    (final_entry_win_pc_df_fold1_p2['Fixed_StopLoss_TakeProfit_testing'] >= 60) & 
    (final_entry_win_pc_df_fold1_p2['Fixed_Bar_testing'] >= 60) & 
    (final_entry_win_pc_df_fold1_p2['Random_Exit_testing'] >= 60)
]['strategy'].tolist()

# finding strategies that have passed entry testing and have winning percentage equal to 60 
# or more (except for Random_Entry_testing) for exit testing cases
exit_testing_strategies = final_exit_win_pc_df_fold1_p2[
    (final_exit_win_pc_df_fold1_p2['Trend_testing'] >= 60) & 
    (final_exit_win_pc_df_fold1_p2['Countertrend_testing'] >= 60) & 
    (final_exit_win_pc_df_fold1_p2['Random_Entry_testing'] >= 40) &
    (final_exit_win_pc_df_fold1_p2['strategy'].isin(entry_testing_strategies))
]['strategy'].tolist()

# finding strategies that have passed both entry and exit testing and 
# have winning percentage equal to 60 or more for core testing cases
core_testing_strategies1 = final_core_win_pc_df_fold1_p2[
    (final_core_win_pc_df_fold1_p2['Core_Testing'] >= 60) &
    (final_core_win_pc_df_fold1_p2['strategy'].isin(exit_testing_strategies))
]['strategy'].tolist()

# finding strategies that have passed entry testing and 
# have winning percentage equal to 60 or more for core testing cases
core_testing_strategies2 = final_core_win_pc_df_fold1_p2[
    (final_core_win_pc_df_fold1_p2['Core_Testing'] >= 60) &
    (final_core_win_pc_df_fold1_p2['strategy'].isin(entry_testing_strategies))
]['strategy'].tolist()

# finding performance for strategies that have passed all tests and have positive ROI
perf_testing_strategies1 = final_perf_df_fold1_p2[
    (final_perf_df_fold1_p2['strategy'].isin(core_testing_strategies1)) & 
    (final_perf_df_fold1_p2['ROI (%)'] > 0) & 
    (final_perf_df_fold1_p2['Drawdown (%)'] < 100)
]['strategy'].tolist()

# finding performance for strategies that have passed entry testing, core testing, and have positive ROI
perf_testing_strategies2 = final_perf_df_fold1_p2[
    (final_perf_df_fold1_p2['strategy'].isin(core_testing_strategies2)) & 
    (final_perf_df_fold1_p2['ROI (%)'] > 0) & 
    (final_perf_df_fold1_p2['Drawdown (%)'] < 100)
]['strategy'].tolist()

len(perf_testing_strategies1), len(perf_testing_strategies2)

(0, 0)

In [23]:
df_str_fold1_p2 = df_str_fold1_p2[df_str_fold1_p2['strategy'].isin(perf_testing_strategies2)]
df_str_fold1_p2.drop(columns='strategy', inplace=True)
df_str_fold1_p2

,buy,exit_buy,sell,exit_sell,fitness


In [24]:
df_str = pd.concat([df_str_fold1_p1, df_str_fold1_p2], axis=0)
df_str.reset_index(inplace=True, drop=True)
df_str

,buy,exit_buy,sell,exit_sell,fitness
0,(price_data['googl_close'][MAX_LAG:] > get_lag...,"(get_lag(price_data['aapl_close'], lag=8)[MAX_...",((numba_indicators.moving_average(prices=price...,"(get_lag(price_data['aapl_close'], lag=8)[MAX_...",-1722.437713
1,(price_data['googl_close'][MAX_LAG:] > get_lag...,"(get_lag(price_data['aapl_close'], lag=8)[MAX_...",((numba_indicators.moving_average(prices=price...,"(get_lag(price_data['aapl_close'], lag=8)[MAX_...",-1631.689868
2,((numba_indicators.moving_average(prices=price...,(price_data['day_of_week'][MAX_LAG:] >= 4),(numba_indicators.ultimate_oscillator(high=pri...,(price_data['day_of_week'][MAX_LAG:] >= 4),-1471.767035
3,((numba_indicators.moving_average(prices=price...,(price_data['day_of_week'][MAX_LAG:] >= 4),(numba_indicators.ultimate_oscillator(high=pri...,(price_data['day_of_week'][MAX_LAG:] >= 4),-1146.334724


In [25]:
df_str.to_csv('data_folds_1min/filtered_strategies_for_fold1.csv', index=False)

## Testing on out of fold data

In [ ]:
df_str = pd.read_csv('data_folds_1min/filtered_strategies_for_fold1.csv')
data_path = Path(r'C:/\Users/\vchar/\OneDrive/\Desktop/\ML Projects/\Upwork/\AlgoT_ML_Dev/\GrammarEvolution/\PonyGE2/\all_data_1min_all.csv')

final_entry_win_pc_df = pd.DataFrame()
final_exit_win_pc_df = pd.DataFrame()
final_core_win_pc_df = pd.DataFrame()
final_perf_df = pd.DataFrame()
final_mc_df = pd.DataFrame()

equity_curve_dict = {}

# bars_per_5week = 7 * 60 * 24 * 5
# n_bars_per_year = 7 * 60 * 24 * 52

# if df_52w.shape[0] < n_bars_per_year:
#     n_bars_per_year = df_52w.shape[0]

lag_txt = '{i}'

strategy_idx = 1

for row in tqdm(df_str.itertuples()):#df_str.iloc[:10].itertuples()

    buy_signal_txt = row.buy
    buy_exit_txt = row.exit_buy
    sell_signal_txt = row.sell
    sell_exit_txt = row.exit_sell

    text_code = f'''import os
CUR_DIR = os.getcwd()
os.chdir('src')
#import pandas as pd
import numpy as np
import gc
from fitness.indicators import numba_indicators, signals
from fitness.performance.helper_func import merge_buy_sell_pnl, get_drawdowns, get_pnl, get_lag
from fitness.performance.helper_func import trading_signals_buy, trading_signals_sell, change_exit
os.chdir(CUR_DIR)
#from numba import njit
COMMISSION = 0.015
SLIPPAGE = 0.00005
AVAILABLE_CAPITAL = 700000
TRADE_SIZE = 0.5
MAX_LAG = 99
buy_idxs, buy_exit_idxs = trading_signals_buy(buy_signal={buy_signal_txt}, exit_signal={buy_exit_txt})
sell_idxs, sell_exit_idxs = trading_signals_sell(sell_signal={sell_signal_txt}, exit_signal={sell_exit_txt})
# if (len(buy_idxs) == 0 or len(buy_exit_idxs) == 0) and (len(sell_idxs) == 0 or len(sell_exit_idxs) == 0):
#     fitness = -9999999
#     avg_drawdown = -9999999
# else:
try:
    buy_idxs, buy_exit_idxs, sell_idxs, sell_exit_idxs = change_exit(buy_idxs, buy_exit_idxs, sell_idxs, sell_exit_idxs)
except:
    pass
if (len(buy_idxs) == 0 or len(buy_exit_idxs) == 0) and (len(sell_idxs) == 0 or len(sell_exit_idxs) == 0):
    fitness = -9999999
    avg_drawdown = -9999999
else:
    buy_idxs = np.array(buy_idxs)
    sell_idxs = np.array(sell_idxs)
    open_prices = price_data['btc_open']
    # pnl_mren_arr, max_dd_mren_arr = get_monkey_test_results(open_prices, buy_idxs, sell_idxs, COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE)
    buy_prices = open_prices[np.isin(np.arange(len(open_prices)), buy_idxs)]
    buy_exit_prices = open_prices[np.isin(np.arange(len(open_prices)), buy_exit_idxs)]
    sell_prices = open_prices[np.isin(np.arange(len(open_prices)), sell_idxs)]
    sell_exit_prices = open_prices[np.isin(np.arange(len(open_prices)), sell_exit_idxs)]
    buy_arr = get_pnl(buy_exit_prices, buy_prices, COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE, 1)
    buy_pnl = np.sum(buy_arr)
    sell_arr = get_pnl(sell_exit_prices, sell_prices, COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE, 0)
    sell_pnl = np.sum(sell_arr)
    all_arr = merge_buy_sell_pnl(buy_idxs, sell_idxs, buy_arr, sell_arr)
    total_pnl = buy_pnl + sell_pnl
    equity_curve_arr = np.cumsum(all_arr)
    drawdowns = get_drawdowns(equity_curve_arr)
    if len(drawdowns[drawdowns!=0]) != 0:
        avg_drawdown = np.sum(drawdowns[drawdowns!=0]) / len(drawdowns[drawdowns!=0])
        fitness = total_pnl / avg_drawdown
    elif total_pnl <= 0 or len(drawdowns[drawdowns!=0]) == 0:
        fitness = -9999999
        avg_drawdown = -9999999
gc.collect()'''
    
    entry_test_n_not_worked = 0
    exit_test_n_not_worked = 0
    core_test_n_not_worked = 0
    perf_n_not_worked = 0
    mc_n_not_worked = 0
    n_total_cases = 0

    equity_curve_dict = defaultdict(list)

    entry_walk_forward_dict = defaultdict(list)

    exit_walk_forward_dict = defaultdict(list)

    core_walk_forward_dict = defaultdict(list)

    performance_walk_forward_dict = defaultdict(list)

    mc_walk_forward_dict = defaultdict(list)

    for n_fold in range(2, 10):

        n_total_cases += 1

        # df = df_52w.iloc[idx:idx+bars_per_5week, :]
        # df.reset_index(drop=True, inplace=True)
        df = generate_fold_data(data_path, fold=n_fold)
        # df.reset_index(drop=True, inplace=True)

        price_data = {}
        for col in df.columns:
            if col == 'datetime':
                continue
            else:
                price_data[col] = df[col].values
        price_data['day_of_week'] = (df['datetime'].dt.dayofweek + 1).values
        price_data['month'] = df['datetime'].dt.month.values

        exec_dict = {'price_data': price_data}
        exec(text_code, exec_dict)

        try:
            equity_curve_arr = exec_dict['equity_curve_arr']
            equity_curve_dict[strategy_idx].append(equity_curve_arr)
            print(f"fold{n_fold}: {len(equity_curve_arr)}")
        except:
            pass

        try:
            fixed_winning_percent, fixed_bar_winning_percent, random_winning_percent = calculate_mean_win_perc_entry_testing(exec_dict, df)
            entry_walk_forward_dict['fixed_sp_testing'].append(fixed_winning_percent)
            entry_walk_forward_dict['fixed_bar_testing'].append(fixed_bar_winning_percent)
            entry_walk_forward_dict['random_exit_testing'].append(random_winning_percent)
        except:
            entry_test_n_not_worked += 1

        try:
            trend_winning_percent, countertrend_winning_percent, random_winning_percent = calculate_mean_win_perc_exit_testing(exec_dict, df)
            exit_walk_forward_dict['trend_entry_testing'].append(trend_winning_percent)
            exit_walk_forward_dict['countertrend_entry_testing'].append(countertrend_winning_percent)
            exit_walk_forward_dict['random_entry_testing'].append(random_winning_percent)
        except:
            exit_test_n_not_worked += 1

        try:
            winning_percent = calculate_mean_win_perc_core_testing(exec_dict)
            core_walk_forward_dict['core_testing'].append(winning_percent)
        except:
            core_test_n_not_worked += 1

        try:
            metric_dict = calculate_mean_performance(exec_dict, monkey_test=False)
            performance_walk_forward_dict['n_trades'].append(metric_dict['n_trades'])
            performance_walk_forward_dict['pnl'].append(metric_dict['overall_pnl'])
            performance_walk_forward_dict['roi'].append(metric_dict['roi'])
            performance_walk_forward_dict['avg_drawdown'].append(metric_dict['avg_drawdown'])
            performance_walk_forward_dict['drawdown'].append(metric_dict['max_dd'])
            performance_walk_forward_dict['drawdown_dur'].append(metric_dict['drawdown_dur'])
            performance_walk_forward_dict['pnl_avgd_ratio'].append(metric_dict['pnl_avgd_ratio'])
            performance_walk_forward_dict['sharpe_ratio'].append(metric_dict['sharpe_ratio'])
            performance_walk_forward_dict['sortino_ratio'].append(metric_dict['sortino_ratio'])
            if 'mt_pnl' in metric_dict.keys():
                performance_walk_forward_dict['mt_pnl'].append(metric_dict['mt_pnl'])
                performance_walk_forward_dict['mt_mdd'].append(metric_dict['mt_mdd'])
        except:
            perf_n_not_worked += 1

        try:
            mc_dict = calculate_mc_performance(exec_dict)
            mc_walk_forward_dict['median_max_dd'].append(mc_dict['median_max_dd'])
            mc_walk_forward_dict['median_dd_dur'].append(mc_dict['median_dd_dur'])
            mc_walk_forward_dict['median_profit'].append(mc_dict['median_profit'])
            mc_walk_forward_dict['median_return'].append(mc_dict['median_return'])
            mc_walk_forward_dict['return_dd_ratio'].append(mc_dict['return_dd_ratio'])
            mc_walk_forward_dict['prob_profit'].append(mc_dict['prob_profit'])
        except:
            mc_n_not_worked += 1

    temp_signal_df = pd.DataFrame(
        {
            'strategy': f'strategy{strategy_idx}', 
            'buy': [buy_signal_txt], 
            'sell': [sell_signal_txt],
            'exit_buy': [buy_exit_txt], 
            'exit_sell': [sell_exit_txt]
        }
    )

    entry_win_pc_df = get_entry_win_pc_df(entry_walk_forward_dict, entry_test_n_not_worked, n_total_cases)
    entry_win_pc_df = pd.concat([temp_signal_df, entry_win_pc_df], axis=1)
    final_entry_win_pc_df = pd.concat([final_entry_win_pc_df, entry_win_pc_df])

    exit_win_pc_df = get_exit_win_pc_df(exit_walk_forward_dict, exit_test_n_not_worked, n_total_cases)
    exit_win_pc_df = pd.concat([temp_signal_df, exit_win_pc_df], axis=1)
    final_exit_win_pc_df = pd.concat([final_exit_win_pc_df, exit_win_pc_df])

    core_win_pc_df = get_core_win_pc_df(core_walk_forward_dict, core_test_n_not_worked, n_total_cases)
    core_win_pc_df = pd.concat([temp_signal_df, core_win_pc_df], axis=1)
    final_core_win_pc_df = pd.concat([final_core_win_pc_df, core_win_pc_df])

    perf_df = get_perf_df(performance_walk_forward_dict, perf_n_not_worked, n_total_cases)
    perf_df = pd.concat([temp_signal_df, perf_df], axis=1)
    final_perf_df = pd.concat([final_perf_df, perf_df])

    mc_df = get_mc_df(mc_walk_forward_dict, mc_n_not_worked, n_total_cases)
    mc_df = pd.concat([temp_signal_df, mc_df], axis=1)
    final_mc_df = pd.concat([final_mc_df, mc_df])

    strategy_idx += 1

0it [00:00, ?it/s]

fold2: 8
fold3: 1
fold4: 7
fold5: 6
fold6: 3
fold7: 7
fold8: 7
fold9: 2


1it [00:53, 53.65s/it]

fold2: 8
fold3: 2
fold4: 8
fold5: 6
fold6: 7
fold7: 6
fold8: 13
fold9: 3


2it [01:56, 58.83s/it]

fold2: 5
fold3: 4
fold4: 4
fold5: 5
fold6: 4
fold7: 3
fold8: 4
fold9: 3


3it [03:02, 62.19s/it]

fold2: 5
fold3: 4
fold4: 5
fold5: 3
fold6: 4
fold7: 3
fold8: 3
fold9: 3


4it [04:07, 61.94s/it]


In [29]:
equity_curve_dict.keys()

dict_keys([4])

In [30]:
final_entry_win_pc_df

,strategy,buy,sell,Fixed_StopLoss_TakeProfit_testing,Fixed_Bar_testing,Random_Exit_testing,Not_Working
0,strategy1,(price_data['googl_close'][MAX_LAG:] > get_lag...,((numba_indicators.moving_average(prices=price...,40.042093,51.070679,49.092261,0.0
0,strategy2,(price_data['googl_close'][MAX_LAG:] > get_lag...,((numba_indicators.moving_average(prices=price...,27.228696,69.146867,56.859998,0.0
0,strategy3,((numba_indicators.moving_average(prices=price...,(numba_indicators.ultimate_oscillator(high=pri...,NaN,NaN,NaN,0.0
0,strategy4,((numba_indicators.moving_average(prices=price...,(numba_indicators.ultimate_oscillator(high=pri...,31.293958,29.674276,43.196222,0.0


In [31]:
final_exit_win_pc_df

,strategy,buy,sell,Trend_testing,Countertrend_testing,Random_Entry_testing,Not_Working
0,strategy1,(price_data['googl_close'][MAX_LAG:] > get_lag...,((numba_indicators.moving_average(prices=price...,60.728747,60.272854,47.211374,0.0
0,strategy2,(price_data['googl_close'][MAX_LAG:] > get_lag...,((numba_indicators.moving_average(prices=price...,60.730732,60.297911,47.212092,0.0
0,strategy3,((numba_indicators.moving_average(prices=price...,(numba_indicators.ultimate_oscillator(high=pri...,60.730484,60.292452,47.021500,0.0
0,strategy4,((numba_indicators.moving_average(prices=price...,(numba_indicators.ultimate_oscillator(high=pri...,60.730732,60.293197,46.787665,0.0


In [32]:
final_core_win_pc_df

,strategy,buy,sell,Core_Testing,Not_Working
0,strategy1,(price_data['googl_close'][MAX_LAG:] > get_lag...,((numba_indicators.moving_average(prices=price...,80.357143,0.0
0,strategy2,(price_data['googl_close'][MAX_LAG:] > get_lag...,((numba_indicators.moving_average(prices=price...,78.663004,0.0
0,strategy3,((numba_indicators.moving_average(prices=price...,(numba_indicators.ultimate_oscillator(high=pri...,46.458333,0.0
0,strategy4,((numba_indicators.moving_average(prices=price...,(numba_indicators.ultimate_oscillator(high=pri...,51.875000,0.0


In [33]:
final_perf_df

,strategy,buy,sell,N_Trades,PNL,ROI (%),AVG_Drawdown,Drawdown (%),Drawdown_Duration,PNL_AVGD_Ratio,Sharpe_Ratio,Sortino_Ratio,Not_Working
0,strategy1,(price_data['googl_close'][MAX_LAG:] > get_lag...,((numba_indicators.moving_average(prices=price...,4.125,9450.869521,2.700248,-6.249895e+06,72.182374,2.250,-6.249960e+06,2.951300e+09,3922.246825,0.0
0,strategy2,(price_data['googl_close'][MAX_LAG:] > get_lag...,((numba_indicators.moving_average(prices=price...,5.625,10033.360627,2.866674,-4.998660e+06,136.141708,3.875,-4.999991e+06,5.831904e+02,1011.751198,0.0
0,strategy3,((numba_indicators.moving_average(prices=price...,(numba_indicators.ultimate_oscillator(high=pri...,3.000,-3847.247414,-1.099214,-7.497449e+06,147.355605,1.625,-7.499999e+06,3.427127e+01,-49.492249,0.0
0,strategy4,((numba_indicators.moving_average(prices=price...,(numba_indicators.ultimate_oscillator(high=pri...,2.750,8491.543634,2.426155,-6.247061e+06,184.409624,1.125,-6.249994e+06,-1.554754e+01,180.651450,0.0


In [ ]:
final_mc_df

,strategy,buy,sell,median_drawdown (%),median_drawdown_duration,median_profit,median_ROI (%),ratio,prob,Not_Working
0,strategy1,(price_data['googl_close'][MAX_LAG:] > get_lag...,((numba_indicators.moving_average(prices=price...,0.804643,0.500,9163.095925,2.618027,NaN,0.874525,0.0
0,strategy2,(price_data['googl_close'][MAX_LAG:] > get_lag...,((numba_indicators.moving_average(prices=price...,17.611540,1.500,9809.456751,2.802702,NaN,0.822137,0.0
0,strategy3,((numba_indicators.moving_average(prices=price...,(numba_indicators.ultimate_oscillator(high=pri...,6.481421,1.625,-3819.773210,-1.091364,NaN,0.478062,0.0
0,strategy4,((numba_indicators.moving_average(prices=price...,(numba_indicators.ultimate_oscillator(high=pri...,0.815417,1.375,8356.348090,2.387528,NaN,0.561850,0.0


In [42]:
print(f"Buy: {df_str.iloc[0]['buy']}")
print(f"Buy exit: {df_str.iloc[0]['exit_buy']}")
print(f"Sell: {df_str.iloc[0]['sell']}")
print(f"Sell exit: {df_str.iloc[0]['exit_sell']}")

Buy: (price_data['googl_close'][MAX_LAG:] > get_lag(price_data['coin_close'], lag=77)[MAX_LAG:])
Buy exit: (get_lag(price_data['aapl_close'], lag=8)[MAX_LAG:] <= numba_indicators.on_balance_volume(price=price_data['gc_close'], volume=price_data['xrp_volume'])[MAX_LAG:])
Sell: ((numba_indicators.moving_average(prices=price_data['btc_close'], window=97)[MAX_LAG:] < numba_indicators.moving_average(prices=price_data['btc_close'], window=104)[MAX_LAG:]) & (price_data['btc_close'][MAX_LAG:] > (price_data['btc_high'][MAX_LAG:] - 339)) & (get_lag(numba_indicators.moving_average(prices=price_data['btc_close'], window=59), lag=1)[MAX_LAG:] > get_lag(numba_indicators.moving_average(prices=price_data['btc_close'], window=3720), lag=1)[MAX_LAG:]))
Sell exit: (get_lag(price_data['aapl_close'], lag=8)[MAX_LAG:] <= numba_indicators.on_balance_volume(price=price_data['gc_close'], volume=price_data['xrp_volume'])[MAX_LAG:])


# Loading results for fold2: from 

In [3]:
final_entry_win_pc_df_fold2_p1 = pd.read_csv('testing_results/entry_testing_fold2_p1.csv')
final_exit_win_pc_df_fold2_p1 = pd.read_csv('testing_results/exit_testing_fold2_p1.csv')
final_core_win_pc_df_fold2_p1 = pd.read_csv('testing_results/core_testing_fold2_p1.csv')
final_perf_df_fold2_p1 = pd.read_csv('testing_results/perf_fold2_p1.csv')
final_mc_df_fold2_p1 = pd.read_csv('testing_results/mc_fold2_p1.csv')

final_entry_win_pc_df_fold2_p2 = pd.read_csv('testing_results/entry_testing_fold2_p2.csv')
final_exit_win_pc_df_fold2_p2 = pd.read_csv('testing_results/exit_testing_fold2_p2.csv')
final_core_win_pc_df_fold2_p2 = pd.read_csv('testing_results/core_testing_fold2_p2.csv')
final_perf_df_fold2_p2 = pd.read_csv('testing_results/perf_fold2_p2.csv')
final_mc_df_fold2_p2 = pd.read_csv('testing_results/mc_fold2_p2.csv')

strategy_file_path = Path(r"C:/\Users/\vchar/\Downloads/\ge_results (6).csv")

try:
    df_str_fold2_p1 = pd.read_csv(strategy_file_path)
except:
    df_str_fold2_p1 = pd.read_csv(strategy_file_path, sep=';')
    
df_str_fold2_p1 = df_str_fold2_p1[(df_str_fold2_p1['fitness'] < 0) & (df_str_fold2_p1['fitness'] <= -1000)]
df_str_fold2_p1 = df_str_fold2_p1[~df_str_fold2_p1.duplicated()]
df_str_fold2_p1.sort_values('fitness', ascending=True, inplace=True)
df_str_fold2_p1.reset_index(drop=True, inplace=True)
df_str_fold2_p1['strategy'] = list(map(lambda x: f"strategy{x}", np.arange(1, df_str_fold2_p1.shape[0]+1)))

strategy_file_path = Path(r"C:/\Users/\vchar/\Downloads/\ge_results (6).csv")

try:
    df_str_fold2_p2 = pd.read_csv(strategy_file_path)
except:
    df_str_fold2_p2 = pd.read_csv(strategy_file_path, sep=';')

df_str_fold2_p2 = df_str_fold2_p2[(df_str_fold2_p2['fitness'] < 0) & (df_str_fold2_p2['fitness'] > -1000)]
df_str_fold2_p2 = df_str_fold2_p2[~df_str_fold2_p2.duplicated()]
df_str_fold2_p2.sort_values('fitness', ascending=True, inplace=True)
df_str_fold2_p2.reset_index(drop=True, inplace=True)
df_str_fold2_p2 = df_str_fold2_p2.iloc[:5000]
df_str_fold2_p2['strategy'] = list(map(lambda x: f"strategy{x}", np.arange(1, df_str_fold2_p2.shape[0]+1)))

# Analysing results for fold2: from

## Part1

In [4]:
# finding strategies that have winning percentage equal to 60 or more for entry testing cases
entry_testing_strategies = final_entry_win_pc_df_fold2_p1[
    (final_entry_win_pc_df_fold2_p1['Fixed_StopLoss_TakeProfit_testing'] >= 60) & 
    (final_entry_win_pc_df_fold2_p1['Fixed_Bar_testing'] >= 60) & 
    (final_entry_win_pc_df_fold2_p1['Random_Exit_testing'] >= 60)
]['strategy'].tolist()

# finding strategies that have passed entry testing and have winning percentage equal to 60 
# or more (except for Random_Entry_testing) for exit testing cases
exit_testing_strategies = final_exit_win_pc_df_fold2_p1[
    (final_exit_win_pc_df_fold2_p1['Trend_testing'] >= 60) & 
    (final_exit_win_pc_df_fold2_p1['Countertrend_testing'] >= 60) & 
    (final_exit_win_pc_df_fold2_p1['Random_Entry_testing'] >= 40) &
    (final_exit_win_pc_df_fold2_p1['strategy'].isin(entry_testing_strategies))
]['strategy'].tolist()

# finding strategies that have passed both entry and exit testing and 
# have winning percentage equal to 60 or more for core testing cases
core_testing_strategies1 = final_core_win_pc_df_fold2_p1[
    (final_core_win_pc_df_fold2_p1['Core_Testing'] >= 60) &
    (final_core_win_pc_df_fold2_p1['strategy'].isin(exit_testing_strategies))
]['strategy'].tolist()

# finding strategies that have passed entry testing and 
# have winning percentage equal to 60 or more for core testing cases
core_testing_strategies2 = final_core_win_pc_df_fold2_p1[
    (final_core_win_pc_df_fold2_p1['Core_Testing'] >= 60) &
    (final_core_win_pc_df_fold2_p1['strategy'].isin(entry_testing_strategies))
]['strategy'].tolist()

# finding performance for strategies that have passed all tests and have positive ROI
perf_testing_strategies1 = final_perf_df_fold2_p1[
    (final_perf_df_fold2_p1['strategy'].isin(core_testing_strategies1)) & 
    (final_perf_df_fold2_p1['ROI (%)'] > 0) & 
    (final_perf_df_fold2_p1['Drawdown (%)'] < 100)
]['strategy'].tolist()

# finding performance for strategies that have passed entry testing, core testing, and have positive ROI
perf_testing_strategies2 = final_perf_df_fold2_p1[
    (final_perf_df_fold2_p1['strategy'].isin(core_testing_strategies2)) & 
    (final_perf_df_fold2_p1['ROI (%)'] > 0) & 
    (final_perf_df_fold2_p1['Drawdown (%)'] < 100)
]['strategy'].tolist()

len(perf_testing_strategies1), len(perf_testing_strategies2)

(5, 5)

In [5]:
df_str_fold2_p1 = df_str_fold2_p1[df_str_fold2_p1['strategy'].isin(perf_testing_strategies2)]
df_str_fold2_p1.drop(columns='strategy', inplace=True)
df_str_fold2_p1

,buy,exit_buy,sell,exit_sell,fitness
5007,((numba_indicators.relative_strength_index(pri...,(price_data['btc_close'][MAX_LAG:] == signals....,((price_data['day_of_week'][MAX_LAG:] == 5) & ...,(price_data['btc_close'][MAX_LAG:] == signals....,-11599.263196
5008,((numba_indicators.relative_strength_index(pri...,(price_data['btc_close'][MAX_LAG:] == signals....,((price_data['day_of_week'][MAX_LAG:] == 5) & ...,(price_data['btc_close'][MAX_LAG:] == signals....,-11599.263196
5015,((numba_indicators.relative_strength_index(pri...,(price_data['btc_close'][MAX_LAG:] == signals....,((price_data['day_of_week'][MAX_LAG:] == 5) & ...,(price_data['btc_close'][MAX_LAG:] == signals....,-11560.010144
7514,((numba_indicators.moving_average(prices=price...,(price_data['btc_close'][MAX_LAG:] < signals.m...,((numba_indicators.moving_average(prices=price...,(price_data['btc_close'][MAX_LAG:] < signals.m...,-1427.854066
7588,((numba_indicators.moving_average(prices=price...,(price_data['btc_close'][MAX_LAG:] < signals.m...,((price_data['day_of_week'][MAX_LAG:] == 5) & ...,(price_data['btc_close'][MAX_LAG:] < signals.m...,-1392.410478


## Part2

In [6]:
# finding strategies that have winning percentage equal to 60 or more for entry testing cases
entry_testing_strategies = final_entry_win_pc_df_fold2_p2[
    (final_entry_win_pc_df_fold2_p2['Fixed_StopLoss_TakeProfit_testing'] >= 60) & 
    (final_entry_win_pc_df_fold2_p2['Fixed_Bar_testing'] >= 60) & 
    (final_entry_win_pc_df_fold2_p2['Random_Exit_testing'] >= 60)
]['strategy'].tolist()

# finding strategies that have passed entry testing and have winning percentage equal to 60 
# or more (except for Random_Entry_testing) for exit testing cases
exit_testing_strategies = final_exit_win_pc_df_fold2_p2[
    (final_exit_win_pc_df_fold2_p2['Trend_testing'] >= 60) & 
    (final_exit_win_pc_df_fold2_p2['Countertrend_testing'] >= 60) & 
    (final_exit_win_pc_df_fold2_p2['Random_Entry_testing'] >= 40) &
    (final_exit_win_pc_df_fold2_p2['strategy'].isin(entry_testing_strategies))
]['strategy'].tolist()

# finding strategies that have passed both entry and exit testing and 
# have winning percentage equal to 60 or more for core testing cases
core_testing_strategies1 = final_core_win_pc_df_fold2_p2[
    (final_core_win_pc_df_fold2_p2['Core_Testing'] >= 60) &
    (final_core_win_pc_df_fold2_p2['strategy'].isin(exit_testing_strategies))
]['strategy'].tolist()

# finding strategies that have passed entry testing and 
# have winning percentage equal to 60 or more for core testing cases
core_testing_strategies2 = final_core_win_pc_df_fold2_p2[
    (final_core_win_pc_df_fold2_p2['Core_Testing'] >= 60) &
    (final_core_win_pc_df_fold2_p2['strategy'].isin(entry_testing_strategies))
]['strategy'].tolist()

# finding performance for strategies that have passed all tests and have positive ROI
perf_testing_strategies1 = final_perf_df_fold2_p2[
    (final_perf_df_fold2_p2['strategy'].isin(core_testing_strategies1)) & 
    (final_perf_df_fold2_p2['ROI (%)'] > 0) & 
    (final_perf_df_fold2_p2['Drawdown (%)'] < 100)
]['strategy'].tolist()

# finding performance for strategies that have passed entry testing, core testing, and have positive ROI
perf_testing_strategies2 = final_perf_df_fold2_p2[
    (final_perf_df_fold2_p2['strategy'].isin(core_testing_strategies2)) & 
    (final_perf_df_fold2_p2['ROI (%)'] > 0) & 
    (final_perf_df_fold2_p2['Drawdown (%)'] < 100)
]['strategy'].tolist()

len(perf_testing_strategies1), len(perf_testing_strategies2)

(2, 2)

In [7]:
df_str_fold2_p2 = df_str_fold2_p2[df_str_fold2_p2['strategy'].isin(perf_testing_strategies2)]
df_str_fold2_p2.drop(columns='strategy', inplace=True)
df_str_fold2_p2

,buy,exit_buy,sell,exit_sell,fitness
2103,(numba_indicators.adx(high=price_data['btc_hig...,(price_data['btc_close'][MAX_LAG:] < signals.m...,((price_data['day_of_week'][MAX_LAG:] == 5) & ...,(price_data['btc_close'][MAX_LAG:] < signals.m...,-751.829209
2116,(numba_indicators.adx(high=price_data['btc_hig...,(price_data['btc_close'][MAX_LAG:] < signals.m...,((price_data['day_of_week'][MAX_LAG:] == 5) & ...,(price_data['btc_close'][MAX_LAG:] < signals.m...,-750.768062


In [8]:
df_str = pd.concat([df_str_fold2_p1, df_str_fold2_p2], axis=0)
df_str.reset_index(inplace=True, drop=True)
df_str

,buy,exit_buy,sell,exit_sell,fitness
0,((numba_indicators.relative_strength_index(pri...,(price_data['btc_close'][MAX_LAG:] == signals....,((price_data['day_of_week'][MAX_LAG:] == 5) & ...,(price_data['btc_close'][MAX_LAG:] == signals....,-11599.263196
1,((numba_indicators.relative_strength_index(pri...,(price_data['btc_close'][MAX_LAG:] == signals....,((price_data['day_of_week'][MAX_LAG:] == 5) & ...,(price_data['btc_close'][MAX_LAG:] == signals....,-11599.263196
2,((numba_indicators.relative_strength_index(pri...,(price_data['btc_close'][MAX_LAG:] == signals....,((price_data['day_of_week'][MAX_LAG:] == 5) & ...,(price_data['btc_close'][MAX_LAG:] == signals....,-11560.010144
3,((numba_indicators.moving_average(prices=price...,(price_data['btc_close'][MAX_LAG:] < signals.m...,((numba_indicators.moving_average(prices=price...,(price_data['btc_close'][MAX_LAG:] < signals.m...,-1427.854066
4,((numba_indicators.moving_average(prices=price...,(price_data['btc_close'][MAX_LAG:] < signals.m...,((price_data['day_of_week'][MAX_LAG:] == 5) & ...,(price_data['btc_close'][MAX_LAG:] < signals.m...,-1392.410478
5,(numba_indicators.adx(high=price_data['btc_hig...,(price_data['btc_close'][MAX_LAG:] < signals.m...,((price_data['day_of_week'][MAX_LAG:] == 5) & ...,(price_data['btc_close'][MAX_LAG:] < signals.m...,-751.829209
6,(numba_indicators.adx(high=price_data['btc_hig...,(price_data['btc_close'][MAX_LAG:] < signals.m...,((price_data['day_of_week'][MAX_LAG:] == 5) & ...,(price_data['btc_close'][MAX_LAG:] < signals.m...,-750.768062


In [9]:
df_str.to_csv('data_folds_1min/filtered_strategies_for_fold2.csv', index=False)

## Testing on out of fold data

In [10]:
df_str = pd.read_csv('data_folds_1min/filtered_strategies_for_fold2.csv')
data_path = Path(r'C:/\Users/\vchar/\OneDrive/\Desktop/\ML Projects/\Upwork/\AlgoT_ML_Dev/\GrammarEvolution/\PonyGE2/\all_data_1min_all.csv')

final_entry_win_pc_df = pd.DataFrame()
final_exit_win_pc_df = pd.DataFrame()
final_core_win_pc_df = pd.DataFrame()
final_perf_df = pd.DataFrame()
final_mc_df = pd.DataFrame()

equity_curve_dict = {}

# bars_per_5week = 7 * 60 * 24 * 5
# n_bars_per_year = 7 * 60 * 24 * 52

# if df_52w.shape[0] < n_bars_per_year:
#     n_bars_per_year = df_52w.shape[0]

lag_txt = '{i}'

strategy_idx = 1

for row in tqdm(df_str.itertuples()):#df_str.iloc[:10].itertuples()

    buy_signal_txt = row.buy
    buy_exit_txt = row.exit_buy
    sell_signal_txt = row.sell
    sell_exit_txt = row.exit_sell

    text_code = f'''import os
CUR_DIR = os.getcwd()
os.chdir('src')
#import pandas as pd
import numpy as np
import gc
from fitness.indicators import numba_indicators, signals
from fitness.performance.helper_func import merge_buy_sell_pnl, get_drawdowns, get_pnl, get_lag
from fitness.performance.helper_func import trading_signals_buy, trading_signals_sell, change_exit
os.chdir(CUR_DIR)
#from numba import njit
COMMISSION = 0.015
SLIPPAGE = 0.00005
AVAILABLE_CAPITAL = 700000
TRADE_SIZE = 0.5
MAX_LAG = 99
buy_idxs, buy_exit_idxs = trading_signals_buy(buy_signal={buy_signal_txt}, exit_signal={buy_exit_txt})
sell_idxs, sell_exit_idxs = trading_signals_sell(sell_signal={sell_signal_txt}, exit_signal={sell_exit_txt})
# if (len(buy_idxs) == 0 or len(buy_exit_idxs) == 0) and (len(sell_idxs) == 0 or len(sell_exit_idxs) == 0):
#     fitness = -9999999
#     avg_drawdown = -9999999
# else:
try:
    buy_idxs, buy_exit_idxs, sell_idxs, sell_exit_idxs = change_exit(buy_idxs, buy_exit_idxs, sell_idxs, sell_exit_idxs)
except:
    pass
if (len(buy_idxs) == 0 or len(buy_exit_idxs) == 0) and (len(sell_idxs) == 0 or len(sell_exit_idxs) == 0):
    fitness = np.nan
    avg_drawdown = np.nan
else:
    buy_idxs = np.array(buy_idxs)
    sell_idxs = np.array(sell_idxs)
    open_prices = price_data['btc_open']
    # pnl_mren_arr, max_dd_mren_arr = get_monkey_test_results(open_prices, buy_idxs, sell_idxs, COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE)
    buy_prices = open_prices[np.isin(np.arange(len(open_prices)), buy_idxs)]
    buy_exit_prices = open_prices[np.isin(np.arange(len(open_prices)), buy_exit_idxs)]
    sell_prices = open_prices[np.isin(np.arange(len(open_prices)), sell_idxs)]
    sell_exit_prices = open_prices[np.isin(np.arange(len(open_prices)), sell_exit_idxs)]
    buy_arr = get_pnl(buy_exit_prices, buy_prices, COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE, 1)
    buy_pnl = np.sum(buy_arr)
    sell_arr = get_pnl(sell_exit_prices, sell_prices, COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE, 0)
    sell_pnl = np.sum(sell_arr)
    all_arr = merge_buy_sell_pnl(buy_idxs, sell_idxs, buy_arr, sell_arr)
    total_pnl = buy_pnl + sell_pnl
    equity_curve_arr = np.cumsum(all_arr)
    drawdowns = get_drawdowns(equity_curve_arr)
    if len(drawdowns[drawdowns!=0]) != 0:
        avg_drawdown = np.sum(drawdowns[drawdowns!=0]) / len(drawdowns[drawdowns!=0])
        fitness = total_pnl / avg_drawdown
    else:
        fitness = np.nan
        avg_drawdown = np.nan
gc.collect()'''
    
    entry_test_n_not_worked = 0
    exit_test_n_not_worked = 0
    core_test_n_not_worked = 0
    perf_n_not_worked = 0
    mc_n_not_worked = 0
    n_total_cases = 0

    equity_curve_dict = defaultdict(list)

    entry_walk_forward_dict = defaultdict(list)

    exit_walk_forward_dict = defaultdict(list)

    core_walk_forward_dict = defaultdict(list)

    performance_walk_forward_dict = defaultdict(list)

    mc_walk_forward_dict = defaultdict(list)

    for n_fold in range(1, 10):

        if n_fold == 2:
            continue

        n_total_cases += 1

        # df = df_52w.iloc[idx:idx+bars_per_5week, :]
        # df.reset_index(drop=True, inplace=True)
        df = generate_fold_data(data_path, fold=n_fold)
        # df.reset_index(drop=True, inplace=True)

        price_data = {}
        for col in df.columns:
            if col == 'datetime':
                continue
            else:
                price_data[col] = df[col].values
        price_data['day_of_week'] = (df['datetime'].dt.dayofweek + 1).values
        price_data['month'] = df['datetime'].dt.month.values

        exec_dict = {'price_data': price_data}
        exec(text_code, exec_dict)

        try:
            equity_curve_arr = exec_dict['equity_curve_arr']
            equity_curve_dict[strategy_idx].append(equity_curve_arr)
            print(f"fold{n_fold}: {len(equity_curve_arr)}")
        except:
            pass

        try:
            fixed_winning_percent, fixed_bar_winning_percent, random_winning_percent = calculate_mean_win_perc_entry_testing(exec_dict, df)
            entry_walk_forward_dict['fixed_sp_testing'].append(fixed_winning_percent)
            entry_walk_forward_dict['fixed_bar_testing'].append(fixed_bar_winning_percent)
            entry_walk_forward_dict['random_exit_testing'].append(random_winning_percent)
        except:
            entry_test_n_not_worked += 1

        try:
            trend_winning_percent, countertrend_winning_percent, random_winning_percent = calculate_mean_win_perc_exit_testing(exec_dict, df)
            exit_walk_forward_dict['trend_entry_testing'].append(trend_winning_percent)
            exit_walk_forward_dict['countertrend_entry_testing'].append(countertrend_winning_percent)
            exit_walk_forward_dict['random_entry_testing'].append(random_winning_percent)
        except:
            exit_test_n_not_worked += 1

        try:
            winning_percent = calculate_mean_win_perc_core_testing(exec_dict)
            core_walk_forward_dict['core_testing'].append(winning_percent)
        except:
            core_test_n_not_worked += 1

        try:
            metric_dict = calculate_mean_performance(exec_dict, monkey_test=False)
            performance_walk_forward_dict['n_trades'].append(metric_dict['n_trades'])
            performance_walk_forward_dict['pnl'].append(metric_dict['overall_pnl'])
            performance_walk_forward_dict['roi'].append(metric_dict['roi'])
            performance_walk_forward_dict['avg_drawdown'].append(metric_dict['avg_drawdown'])
            performance_walk_forward_dict['drawdown'].append(metric_dict['max_dd'])
            performance_walk_forward_dict['drawdown_dur'].append(metric_dict['drawdown_dur'])
            performance_walk_forward_dict['pnl_avgd_ratio'].append(metric_dict['pnl_avgd_ratio'])
            performance_walk_forward_dict['sharpe_ratio'].append(metric_dict['sharpe_ratio'])
            performance_walk_forward_dict['sortino_ratio'].append(metric_dict['sortino_ratio'])
            if 'mt_pnl' in metric_dict.keys():
                performance_walk_forward_dict['mt_pnl'].append(metric_dict['mt_pnl'])
                performance_walk_forward_dict['mt_mdd'].append(metric_dict['mt_mdd'])
        except:
            perf_n_not_worked += 1

        try:
            mc_dict = calculate_mc_performance(exec_dict)
            mc_walk_forward_dict['median_max_dd'].append(mc_dict['median_max_dd'])
            mc_walk_forward_dict['median_dd_dur'].append(mc_dict['median_dd_dur'])
            mc_walk_forward_dict['median_profit'].append(mc_dict['median_profit'])
            mc_walk_forward_dict['median_return'].append(mc_dict['median_return'])
            mc_walk_forward_dict['return_dd_ratio'].append(mc_dict['return_dd_ratio'])
            mc_walk_forward_dict['prob_profit'].append(mc_dict['prob_profit'])
        except:
            mc_n_not_worked += 1

    # temp_signal_df = pd.DataFrame({'strategy': f'strategy{strategy_idx}', 'buy': [buy_signal_txt], 'sell': [sell_signal_txt]})

    temp_signal_df = pd.DataFrame(
        {
            'strategy': f'strategy{strategy_idx}', 
            'buy': [buy_signal_txt], 
            'sell': [sell_signal_txt],
            'exit_buy': [buy_exit_txt], 
            'exit_sell': [sell_exit_txt]
        }
    )

    entry_win_pc_df = get_entry_win_pc_df(entry_walk_forward_dict, entry_test_n_not_worked, n_total_cases)
    entry_win_pc_df = pd.concat([temp_signal_df, entry_win_pc_df], axis=1)
    final_entry_win_pc_df = pd.concat([final_entry_win_pc_df, entry_win_pc_df])

    exit_win_pc_df = get_exit_win_pc_df(exit_walk_forward_dict, exit_test_n_not_worked, n_total_cases)
    exit_win_pc_df = pd.concat([temp_signal_df, exit_win_pc_df], axis=1)
    final_exit_win_pc_df = pd.concat([final_exit_win_pc_df, exit_win_pc_df])

    core_win_pc_df = get_core_win_pc_df(core_walk_forward_dict, core_test_n_not_worked, n_total_cases)
    core_win_pc_df = pd.concat([temp_signal_df, core_win_pc_df], axis=1)
    final_core_win_pc_df = pd.concat([final_core_win_pc_df, core_win_pc_df])

    perf_df = get_perf_df(performance_walk_forward_dict, perf_n_not_worked, n_total_cases)
    perf_df = pd.concat([temp_signal_df, perf_df], axis=1)
    final_perf_df = pd.concat([final_perf_df, perf_df])

    mc_df = get_mc_df(mc_walk_forward_dict, mc_n_not_worked, n_total_cases)
    mc_df = pd.concat([temp_signal_df, mc_df], axis=1)
    final_mc_df = pd.concat([final_mc_df, mc_df])

    strategy_idx += 1

0it [00:00, ?it/s]

fold1: 2
fold3: 3
fold4: 3
fold5: 3
fold6: 2
fold7: 5
fold8: 3


1it [00:40, 40.30s/it]

fold9: 4
fold1: 2
fold3: 3
fold4: 3
fold5: 3
fold6: 2
fold7: 5
fold8: 3
fold9: 4


2it [01:23, 42.04s/it]

...

5it [04:11, 53.59s/it]

fold9: 27
fold1: 6
fold3: 3
fold4: 6
fold5: 4
fold6: 3
fold7: 1
fold8: 6
fold9: 2


6it [05:23, 59.66s/it]

fold1: 6
fold3: 3
fold4: 6
fold5: 4
fold6: 3
fold7: 1
fold8: 6
fold9: 2


7it [06:35, 56.57s/it]


In [11]:
final_entry_win_pc_df

,strategy,buy,sell,exit_buy,exit_sell,Fixed_StopLoss_TakeProfit_testing,Fixed_Bar_testing,Random_Exit_testing,Not_Working
0,strategy1,((numba_indicators.relative_strength_index(pri...,((price_data['day_of_week'][MAX_LAG:] == 5) & ...,(price_data['btc_close'][MAX_LAG:] == signals....,(price_data['btc_close'][MAX_LAG:] == signals....,46.855925,23.938614,38.922924,0.0
0,strategy2,((numba_indicators.relative_strength_index(pri...,((price_data['day_of_week'][MAX_LAG:] == 5) & ...,(price_data['btc_close'][MAX_LAG:] == signals....,(price_data['btc_close'][MAX_LAG:] == signals....,46.855925,23.938614,23.939163,0.0
0,strategy3,((numba_indicators.relative_strength_index(pri...,((price_data['day_of_week'][MAX_LAG:] == 5) & ...,(price_data['btc_close'][MAX_LAG:] == signals....,(price_data['btc_close'][MAX_LAG:] == signals....,41.935444,31.711861,38.577071,0.0
0,strategy4,((numba_indicators.moving_average(prices=price...,((numba_indicators.moving_average(prices=price...,(price_data['btc_close'][MAX_LAG:] < signals.m...,(price_data['btc_close'][MAX_LAG:] < signals.m...,44.074171,46.898353,47.924569,0.0
0,strategy5,((numba_indicators.moving_average(prices=price...,((price_data['day_of_week'][MAX_LAG:] == 5) & ...,(price_data['btc_close'][MAX_LAG:] < signals.m...,(price_data['btc_close'][MAX_LAG:] < signals.m...,57.648906,52.678855,56.380457,0.0
0,strategy6,(numba_indicators.adx(high=price_data['btc_hig...,((price_data['day_of_week'][MAX_LAG:] == 5) & ...,(price_data['btc_close'][MAX_LAG:] < signals.m...,(price_data['btc_close'][MAX_LAG:] < signals.m...,52.113848,57.221357,64.723355,0.0
0,strategy7,(numba_indicators.adx(high=price_data['btc_hig...,((price_data['day_of_week'][MAX_LAG:] == 5) & ...,(price_data['btc_close'][MAX_LAG:] < signals.m...,(price_data['btc_close'][MAX_LAG:] < signals.m...,52.113848,57.221357,75.349612,0.0


In [12]:
final_exit_win_pc_df

,strategy,buy,sell,exit_buy,exit_sell,Trend_testing,Countertrend_testing,Random_Entry_testing,Not_Working
0,strategy1,((numba_indicators.relative_strength_index(pri...,((price_data['day_of_week'][MAX_LAG:] == 5) & ...,(price_data['btc_close'][MAX_LAG:] == signals....,(price_data['btc_close'][MAX_LAG:] == signals....,60.567547,60.014979,46.458947,0.0
0,strategy2,((numba_indicators.relative_strength_index(pri...,((price_data['day_of_week'][MAX_LAG:] == 5) & ...,(price_data['btc_close'][MAX_LAG:] == signals....,(price_data['btc_close'][MAX_LAG:] == signals....,60.567547,60.014979,46.929937,0.0
0,strategy3,((numba_indicators.relative_strength_index(pri...,((price_data['day_of_week'][MAX_LAG:] == 5) & ...,(price_data['btc_close'][MAX_LAG:] == signals....,(price_data['btc_close'][MAX_LAG:] == signals....,60.568539,60.014483,46.356620,0.0
0,strategy4,((numba_indicators.moving_average(prices=price...,((numba_indicators.moving_average(prices=price...,(price_data['btc_close'][MAX_LAG:] < signals.m...,(price_data['btc_close'][MAX_LAG:] < signals.m...,60.524632,60.075513,46.794395,0.0
0,strategy5,((numba_indicators.moving_average(prices=price...,((price_data['day_of_week'][MAX_LAG:] == 5) & ...,(price_data['btc_close'][MAX_LAG:] < signals.m...,(price_data['btc_close'][MAX_LAG:] < signals.m...,60.562338,60.032842,47.096609,0.0
0,strategy6,(numba_indicators.adx(high=price_data['btc_hig...,((price_data['day_of_week'][MAX_LAG:] == 5) & ...,(price_data['btc_close'][MAX_LAG:] < signals.m...,(price_data['btc_close'][MAX_LAG:] < signals.m...,60.571764,60.024408,46.869688,0.0
0,strategy7,(numba_indicators.adx(high=price_data['btc_hig...,((price_data['day_of_week'][MAX_LAG:] == 5) & ...,(price_data['btc_close'][MAX_LAG:] < signals.m...,(price_data['btc_close'][MAX_LAG:] < signals.m...,60.568043,60.022175,46.725850,0.0


In [13]:
final_core_win_pc_df

,strategy,buy,sell,exit_buy,exit_sell,Core_Testing,Not_Working
0,strategy1,((numba_indicators.relative_strength_index(pri...,((price_data['day_of_week'][MAX_LAG:] == 5) & ...,(price_data['btc_close'][MAX_LAG:] == signals....,(price_data['btc_close'][MAX_LAG:] == signals....,61.041667,0.0
0,strategy2,((numba_indicators.relative_strength_index(pri...,((price_data['day_of_week'][MAX_LAG:] == 5) & ...,(price_data['btc_close'][MAX_LAG:] == signals....,(price_data['btc_close'][MAX_LAG:] == signals....,61.041667,0.0
0,strategy3,((numba_indicators.relative_strength_index(pri...,((price_data['day_of_week'][MAX_LAG:] == 5) & ...,(price_data['btc_close'][MAX_LAG:] == signals....,(price_data['btc_close'][MAX_LAG:] == signals....,56.875000,0.0
0,strategy4,((numba_indicators.moving_average(prices=price...,((numba_indicators.moving_average(prices=price...,(price_data['btc_close'][MAX_LAG:] < signals.m...,(price_data['btc_close'][MAX_LAG:] < signals.m...,89.637038,0.0
0,strategy5,((numba_indicators.moving_average(prices=price...,((price_data['day_of_week'][MAX_LAG:] == 5) & ...,(price_data['btc_close'][MAX_LAG:] < signals.m...,(price_data['btc_close'][MAX_LAG:] < signals.m...,94.080350,0.0
0,strategy6,(numba_indicators.adx(high=price_data['btc_hig...,((price_data['day_of_week'][MAX_LAG:] == 5) & ...,(price_data['btc_close'][MAX_LAG:] < signals.m...,(price_data['btc_close'][MAX_LAG:] < signals.m...,82.291667,0.0
0,strategy7,(numba_indicators.adx(high=price_data['btc_hig...,((price_data['day_of_week'][MAX_LAG:] == 5) & ...,(price_data['btc_close'][MAX_LAG:] < signals.m...,(price_data['btc_close'][MAX_LAG:] < signals.m...,82.291667,0.0


In [14]:
final_perf_df

,strategy,buy,sell,exit_buy,exit_sell,N_Trades,PNL,ROI (%),AVG_Drawdown,Drawdown (%),Drawdown_Duration,PNL_AVGD_Ratio,Sharpe_Ratio,Sortino_Ratio,Not_Working
0,strategy1,((numba_indicators.relative_strength_index(pri...,((price_data['day_of_week'][MAX_LAG:] == 5) & ...,(price_data['btc_close'][MAX_LAG:] == signals....,(price_data['btc_close'][MAX_LAG:] == signals....,2.125,-3561.522451,-1.017578,29897.954991,447.377203,1.375,0.281696,6.150947e+01,1540.608271,0.0
0,strategy2,((numba_indicators.relative_strength_index(pri...,((price_data['day_of_week'][MAX_LAG:] == 5) & ...,(price_data['btc_close'][MAX_LAG:] == signals....,(price_data['btc_close'][MAX_LAG:] == signals....,2.125,-3561.522451,-1.017578,29897.954991,447.377203,1.375,0.281696,6.150947e+01,1540.608271,0.0
0,strategy3,((numba_indicators.relative_strength_index(pri...,((price_data['day_of_week'][MAX_LAG:] == 5) & ...,(price_data['btc_close'][MAX_LAG:] == signals....,(price_data['btc_close'][MAX_LAG:] == signals....,2.875,-12702.029677,-3.629151,33359.816932,673.056406,1.750,0.036619,-4.946856e+01,729.539571,0.0
0,strategy4,((numba_indicators.moving_average(prices=price...,((numba_indicators.moving_average(prices=price...,(price_data['btc_close'][MAX_LAG:] < signals.m...,(price_data['btc_close'][MAX_LAG:] < signals.m...,102.875,448744.047421,128.212585,319.319614,102.027348,43.125,5336.792269,5.373779e+02,3138.690138,0.0
0,strategy5,((numba_indicators.moving_average(prices=price...,((price_data['day_of_week'][MAX_LAG:] == 5) & ...,(price_data['btc_close'][MAX_LAG:] < signals.m...,(price_data['btc_close'][MAX_LAG:] < signals.m...,20.625,58303.119013,16.658034,708.060407,107.421566,14.000,102.931537,6.070182e+02,1546.227045,0.0
0,strategy6,(numba_indicators.adx(high=price_data['btc_hig...,((price_data['day_of_week'][MAX_LAG:] == 5) & ...,(price_data['btc_close'][MAX_LAG:] < signals.m...,(price_data['btc_close'][MAX_LAG:] < signals.m...,2.875,14487.860404,4.139389,547.304489,89.459423,1.375,17.001160,3.727350e+09,1741.386057,0.0
0,strategy7,(numba_indicators.adx(high=price_data['btc_hig...,((price_data['day_of_week'][MAX_LAG:] == 5) & ...,(price_data['btc_close'][MAX_LAG:] < signals.m...,(price_data['btc_close'][MAX_LAG:] < signals.m...,2.875,15129.475860,4.322707,563.277905,88.953473,1.375,36.637249,3.804116e+09,1047.012017,0.0


In [15]:
final_mc_df

,strategy,buy,sell,exit_buy,exit_sell,median_drawdown (%),median_drawdown_duration,median_profit,median_ROI (%),ratio,prob,Not_Working
0,strategy1,((numba_indicators.relative_strength_index(pri...,((price_data['day_of_week'][MAX_LAG:] == 5) & ...,(price_data['btc_close'][MAX_LAG:] == signals....,(price_data['btc_close'][MAX_LAG:] == signals....,3.709523,1.125,-3552.370339,-1.014963,NaN,0.529925,0.0
0,strategy2,((numba_indicators.relative_strength_index(pri...,((price_data['day_of_week'][MAX_LAG:] == 5) & ...,(price_data['btc_close'][MAX_LAG:] == signals....,(price_data['btc_close'][MAX_LAG:] == signals....,3.701284,1.000,-3556.696530,-1.016199,NaN,0.529125,0.0
0,strategy3,((numba_indicators.relative_strength_index(pri...,((price_data['day_of_week'][MAX_LAG:] == 5) & ...,(price_data['btc_close'][MAX_LAG:] == signals....,(price_data['btc_close'][MAX_LAG:] == signals....,6.218062,1.625,-12636.923409,-3.610550,NaN,0.460800,0.0
0,strategy4,((numba_indicators.moving_average(prices=price...,((numba_indicators.moving_average(prices=price...,(price_data['btc_close'][MAX_LAG:] < signals.m...,(price_data['btc_close'][MAX_LAG:] < signals.m...,5.641082,1.750,446914.568119,127.689877,172.342543,0.987387,0.0
0,strategy5,((numba_indicators.moving_average(prices=price...,((price_data['day_of_week'][MAX_LAG:] == 5) & ...,(price_data['btc_close'][MAX_LAG:] < signals.m...,(price_data['btc_close'][MAX_LAG:] < signals.m...,1.696077,0.500,57178.899002,16.336828,inf,1.000000,0.0
0,strategy6,(numba_indicators.adx(high=price_data['btc_hig...,((price_data['day_of_week'][MAX_LAG:] == 5) & ...,(price_data['btc_close'][MAX_LAG:] < signals.m...,(price_data['btc_close'][MAX_LAG:] < signals.m...,5.810992,0.875,14489.214215,4.139775,inf,0.887775,0.0
0,strategy7,(numba_indicators.adx(high=price_data['btc_hig...,((price_data['day_of_week'][MAX_LAG:] == 5) & ...,(price_data['btc_close'][MAX_LAG:] < signals.m...,(price_data['btc_close'][MAX_LAG:] < signals.m...,7.541044,0.875,15047.516654,4.299290,inf,0.923725,0.0


In [2]:
a = np.array([np.nan, 5, 3, np.nan, 13])
a = [np.nan, 5, 3, np.nan, 13]

np.nanmean(a)

7.0

In [3]:
np.where(np.isnan(a), 1, 0)

array([1, 0, 0, 1, 0])